<font size="6">Code Block 1</font> <br>
<font size="6">Input Required Python Libraries</font> <br>

In [10]:
from swmm_api.input_file import read_inp_file, SwmmInput, section_labels as sections
from swmm_api import read_inp_file, SwmmInput
from swmm_api import read_out_file, SwmmOutput
from swmm_api.output_file import VARIABLES, OBJECTS
from swmm_api import swmm5_run
from swmm_api.input_file.sections import Infiltration
from swmm_api.input_file.sections import SubCatchment
from swmm_api.input_file.sections import Aquifer
from swmm_api.input_file.sections import SubArea
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hydroeval as he
from sklearn.metrics import r2_score 

from swmm_api.input_file import read_inp_file, SwmmInput, section_labels as sections
from swmm_api import read_inp_file, SwmmInput
from swmm_api import read_out_file, SwmmOutput
from swmm_api.output_file import VARIABLES, OBJECTS
from swmm_api import swmm5_run
from swmm_api.input_file.sections import Infiltration
from swmm_api.input_file.sections import SubCatchment
from swmm_api.input_file.sections import Aquifer
from swmm_api.input_file.sections import SubArea
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hydroeval as he

import math
from sklearn import metrics
from sklearn.metrics import r2_score

import datetime


<font size="6">Code Block 2</font> <br>
<font size="6">Input Pre-Loaded Data (See Documentation Page for Proper Formatting):</font> <br>
-SWMM Subbasin Landuse Percentage Composition <br>
-HEC-RAS Steady State Hydraulics Look-up Table <br>
-Hourly Photosynthetically Active Radiation (PAR) <br>
-Estimated Detritus/Leaf Litter Deposition <br>
-Daily Temperature <br>
-Green Infrastructure Parameters

In [12]:
# Input SWMM Subbasin Landuse Composition
sb_lulc_perc_df = pd.read_csv(r"LandUse_Subbasin_Percentages.csv")
sb_lulc_perc_df = sb_lulc_perc_df.set_index('LULC')

#Input HEC-RAS Steady State Hydraulics Look-Up Table
channel_geometry_df = pd.read_csv(r"MC_All_XC_SSHBS_Inputs.csv")
#Sort HEC-RAS Steady State Hydraulic Look-Up Table (if needed)
section_list_UPtoDown = list(channel_geometry_df['Section'])
section_list_UPtoDown = sorted(list(set(section_list_UPtoDown)))
section_list_UPtoDown.reverse()

#Input Hourly PAR Data
hourly_par_df = pd.read_csv(r"MC_HourlyPAR_2021-2022.csv")

#Input Estimated Detritus/Leaf Litter Deposition
detritus_df = pd.read_csv(r"Estimated_Detritus_2021-2022_Calibrated.csv")

#Input Daily Temperature
daily_temp_df = pd.read_csv(r"MC_DailyTEMP_2021-2022.csv")


flow_var_list = []
section_flow_list = []

# Configure Flow Dataframe Columns
for i in section_list_UPtoDown:
    section_flow_list.append(str(i)+' '+'Flow (cms)')
    flow_var_list.append(str(i)+' '+'Flow (cms)')
    flow_var_list.append(str(i)+' '+'Wetted_Perimeter (m)')
    flow_var_list.append(str(i)+' '+'Flow_Area (m2)')
    flow_var_list.append(str(i)+' '+'Flow_Velocity (m/s)')
    flow_var_list.append(str(i)+' '+'Flow_Volume (m3)')
    flow_var_list.append(str(i)+' '+'Benthic_Area (m2)')
    flow_var_list.append(str(i)+' '+'SWMM_TN_Conc (mg/m3)')
    flow_var_list.append(str(i)+' '+'SWMM_TP_Conc (mg/m3)')
    flow_var_list.append(str(i)+' '+'SWMM_TN_Mass (mg)')
    flow_var_list.append(str(i)+' '+'SWMM_TP_Mass (mg)')
    flow_var_list.append(str(i)+' '+'Updated_TN_Mass (mg)')
    flow_var_list.append(str(i)+' '+'Updated_TN_Conc (mg/m3)')
    flow_var_list.append(str(i)+' '+'Updated_TP_Mass (mg)')
    flow_var_list.append(str(i)+' '+'Updated_TP_Conc (mg/m3)')


flow_df = pd.DataFrame(columns = flow_var_list)
#Set Number of Days of simulation (730 = 730 days or Two years)
flow_df['Day'] = detritus_df['Day'][0:730]

#Input SWMM Green Infrastructure Parameters
gi_parameters_df = pd.read_csv(r"LIDPotential_Parameters.csv")


<font size="6">Code Block 3</font> <br>
<font size="6">SWMM Input Configuration and Simulation </font> <br>
-Set groundwater and runoff event mean concentrations based on landuses <br>
-Set Green infrastructure parameters <br>
-Run SWMM Model and Get subbasin outputs for the instream ecosystem model <br>
-Calculate flow and nutrient concentrations from each subbasin

In [7]:

#Set Groundwater and Washoff Event Mean Concentrations for N Based on LandUses
inp = read_inp_file(r"MCSWMM-9-GA_Calibrated_Jun23_CalVal.inp")
commercial_n = 2.0 * 1.27
grasses_n = 1.1 * 1.42
residential_n = 2.0*0.84
roads_n = 2.0*0.73
tree_canopy_n = 0.8*0.6
inp.WASHOFF[('Roads', 'TN1')]['C1'] = inp.WASHOFF[('Roads', 'TN1')]['C1']*1
inp.WASHOFF[('Grasses', 'TN2')]['C1'] = inp.WASHOFF[('Grasses', 'TN2')]['C1']*0.65
inp.WASHOFF[('Residential', 'TN3')]['C1'] = inp.WASHOFF[('Residential', 'TN3')]['C1']*0.5
inp.WASHOFF[('Commercial', 'TN4')]['C1'] = inp.WASHOFF[('Commercial', 'TN4')]['C1']*0.51 
inp.WASHOFF[('TreeCanopy', 'TN5')]['C1'] =inp.WASHOFF[('TreeCanopy', 'TN5')]['C1']*0.64


# Get Area, Width, and Calculate Flow Length and, % imperviousness..... list(range(1,10)) == loop for 9 subbasins
sqft_list = []
for i in list(range(1,10)):
    j = (inp[sections.SUBCATCHMENTS][f'S-{i}']['area']*43560)
    sqft_list.append(j)
    
width_list = []
for i in list(range(1,10)):
    j = inp[sections.SUBCATCHMENTS][f'S-{i}']['width']
    width_list.append(j)

flow_length_list = []
for i in list(range(1,10)):
    j = (inp[sections.SUBCATCHMENTS][f'S-{i}']['area']*43560)/inp[sections.SUBCATCHMENTS][f'S-{i}']['width']
    flow_length_list.append(j)

imp_area_list = []
for i in list(range(1,10)):
    j = (inp[sections.SUBCATCHMENTS][f'S-{i}']['imperviousness']/100)*(inp[sections.SUBCATCHMENTS][f'S-{i}']['area']*43560)
    imp_area_list.append(j)


gi_parameters_df['Subbasin_SqFt'] = sqft_list
gi_parameters_df['Flow_Lengths'] = flow_length_list
gi_parameters_df['Original_Widths'] = width_list
gi_parameters_df['Original_Imps'] = imp_area_list
gi_parameters_df = gi_parameters_df.set_index('Subbasin')


# Set Number of raingardens to implement based on % of residential parcels
sb_1_rg_parcels = int(gi_parameters_df.at[1,'Residential_Parcels']*0)
sb_2_rg_parcels = int(gi_parameters_df.at[2,'Residential_Parcels']*0)
sb_3_rg_parcels = int(gi_parameters_df.at[3,'Residential_Parcels']*0)
sb_4_rg_parcels = int(gi_parameters_df.at[4,'Residential_Parcels']*0)
sb_5_rg_parcels = int(gi_parameters_df.at[5,'Residential_Parcels']*0)
sb_6_rg_parcels = int(gi_parameters_df.at[6,'Residential_Parcels']*0)
sb_7_rg_parcels = int(gi_parameters_df.at[7,'Residential_Parcels']*0)
sb_8_rg_parcels = int(gi_parameters_df.at[8,'Residential_Parcels']*0)
sb_9_rg_parcels = int(gi_parameters_df.at[9,'Residential_Parcels']*0)

total_rg_parcels = sb_1_rg_parcels+sb_2_rg_parcels+sb_3_rg_parcels+sb_4_rg_parcels+sb_5_rg_parcels+sb_6_rg_parcels+sb_7_rg_parcels+sb_8_rg_parcels+sb_9_rg_parcels

#Convert to # of Raingardens to square feet of parcels (10 X 10 m) = 1076.4 square feet
sb_1_rg_area = sb_1_rg_parcels*1076.4
sb_2_rg_area = sb_2_rg_parcels*1076.4
sb_3_rg_area = sb_3_rg_parcels*1076.4
sb_4_rg_area = sb_4_rg_parcels*1076.4
sb_5_rg_area = sb_5_rg_parcels*1076.4
sb_6_rg_area = sb_6_rg_parcels*1076.4
sb_7_rg_area = sb_7_rg_parcels*1076.4
sb_8_rg_area = sb_8_rg_parcels*1076.4
sb_9_rg_area = sb_9_rg_parcels*1076.4

inp[sections.LID_USAGE][('S-1', 'Micro-Bioretention')]['n_replicate'] = sb_1_rg_parcels
inp[sections.LID_USAGE][('S-2', 'Micro-Bioretention')]['n_replicate'] = sb_2_rg_parcels
inp[sections.LID_USAGE][('S-3', 'Micro-Bioretention')]['n_replicate'] = sb_3_rg_parcels
inp[sections.LID_USAGE][('S-4', 'Micro-Bioretention')]['n_replicate'] = sb_4_rg_parcels
inp[sections.LID_USAGE][('S-5', 'Micro-Bioretention')]['n_replicate'] = sb_5_rg_parcels
inp[sections.LID_USAGE][('S-6', 'Micro-Bioretention')]['n_replicate'] = sb_6_rg_parcels
inp[sections.LID_USAGE][('S-7', 'Micro-Bioretention')]['n_replicate'] = sb_7_rg_parcels
inp[sections.LID_USAGE][('S-8', 'Micro-Bioretention')]['n_replicate'] = sb_8_rg_parcels
inp[sections.LID_USAGE][('S-9', 'Micro-Bioretention')]['n_replicate'] = sb_9_rg_parcels

#Set % Impevious Area Treated by Raingardens at each Subbasin
inp[sections.LID_USAGE][('S-1', 'Micro-Bioretention')]['impervious_portion'] = 0
inp[sections.LID_USAGE][('S-2', 'Micro-Bioretention')]['impervious_portion'] = 0
inp[sections.LID_USAGE][('S-3', 'Micro-Bioretention')]['impervious_portion'] = 0
inp[sections.LID_USAGE][('S-4', 'Micro-Bioretention')]['impervious_portion'] = 0
inp[sections.LID_USAGE][('S-5', 'Micro-Bioretention')]['impervious_portion'] = 0
inp[sections.LID_USAGE][('S-6', 'Micro-Bioretention')]['impervious_portion'] = 0
inp[sections.LID_USAGE][('S-7', 'Micro-Bioretention')]['impervious_portion'] = 0
inp[sections.LID_USAGE][('S-8', 'Micro-Bioretention')]['impervious_portion'] = 0
inp[sections.LID_USAGE][('S-9', 'Micro-Bioretention')]['impervious_portion'] = 0

#Set % Pervious Area Treated by Raingardens at each Subbasin
inp[sections.LID_USAGE][('S-1', 'Micro-Bioretention')]['from_pervious'] = 0
inp[sections.LID_USAGE][('S-2', 'Micro-Bioretention')]['from_pervious'] = 0
inp[sections.LID_USAGE][('S-3', 'Micro-Bioretention')]['from_pervious'] = 0
inp[sections.LID_USAGE][('S-4', 'Micro-Bioretention')]['from_pervious'] = 0
inp[sections.LID_USAGE][('S-5', 'Micro-Bioretention')]['from_pervious'] = 0
inp[sections.LID_USAGE][('S-6', 'Micro-Bioretention')]['from_pervious'] = 0
inp[sections.LID_USAGE][('S-7', 'Micro-Bioretention')]['from_pervious'] = 0
inp[sections.LID_USAGE][('S-8', 'Micro-Bioretention')]['from_pervious'] = 0
inp[sections.LID_USAGE][('S-9', 'Micro-Bioretention')]['from_pervious'] = 0


#Set % of Commercial Roof Area (ha) to be replaced by Green Roofs
sb_1_gr_area = gi_parameters_df.at[1,'Commercial Roof Area (ha)']*0
sb_2_gr_area = gi_parameters_df.at[2,'Commercial Roof Area (ha)']*0
sb_3_gr_area = gi_parameters_df.at[3,'Commercial Roof Area (ha)']*0
sb_4_gr_area = gi_parameters_df.at[4,'Commercial Roof Area (ha)']*0
sb_5_gr_area = gi_parameters_df.at[5,'Commercial Roof Area (ha)']*0
sb_6_gr_area = gi_parameters_df.at[6,'Commercial Roof Area (ha)']*0
sb_7_gr_area = gi_parameters_df.at[7,'Commercial Roof Area (ha)']*0
sb_8_gr_area = gi_parameters_df.at[8,'Commercial Roof Area (ha)']*0

total_gr_area = sb_1_gr_area+sb_2_gr_area+sb_3_gr_area+sb_4_gr_area+sb_5_gr_area+sb_6_gr_area+sb_7_gr_area+sb_8_gr_area


inp[sections.LID_USAGE][('S-1', 'GreenRoof')]['area'] =  sb_1_gr_area
inp[sections.LID_USAGE][('S-2', 'GreenRoof')]['area'] =  sb_2_gr_area
inp[sections.LID_USAGE][('S-3', 'GreenRoof')]['area'] =  sb_3_gr_area
inp[sections.LID_USAGE][('S-4', 'GreenRoof')]['area'] =  sb_4_gr_area
inp[sections.LID_USAGE][('S-5', 'GreenRoof')]['area'] =  sb_5_gr_area
inp[sections.LID_USAGE][('S-6', 'GreenRoof')]['area'] =  sb_6_gr_area
inp[sections.LID_USAGE][('S-7', 'GreenRoof')]['area'] =  sb_7_gr_area
inp[sections.LID_USAGE][('S-8', 'GreenRoof')]['area'] =  sb_8_gr_area

            


# Recalculate Subbasin Widths based on Raingarden and Greenroof Implemented Areas
inp[sections.SUBCATCHMENTS]['S-1']['width'] = gi_parameters_df.at[1,'Original_Widths'] - (sb_1_rg_area+sb_1_gr_area)/gi_parameters_df.at[1,'Flow_Lengths']
inp[sections.SUBCATCHMENTS]['S-2']['width'] = gi_parameters_df.at[2,'Original_Widths'] - (sb_2_rg_area+sb_2_gr_area)/gi_parameters_df.at[2,'Flow_Lengths']
inp[sections.SUBCATCHMENTS]['S-3']['width'] = gi_parameters_df.at[3,'Original_Widths'] - (sb_3_rg_area+sb_3_gr_area)/gi_parameters_df.at[3,'Flow_Lengths']
inp[sections.SUBCATCHMENTS]['S-4']['width'] = gi_parameters_df.at[4,'Original_Widths'] - (sb_4_rg_area+sb_4_gr_area)/gi_parameters_df.at[4,'Flow_Lengths']
inp[sections.SUBCATCHMENTS]['S-5']['width'] = gi_parameters_df.at[5,'Original_Widths'] - (sb_5_rg_area+sb_5_gr_area)/gi_parameters_df.at[5,'Flow_Lengths']
inp[sections.SUBCATCHMENTS]['S-6']['width'] = gi_parameters_df.at[6,'Original_Widths'] - (sb_6_rg_area+sb_6_gr_area)/gi_parameters_df.at[6,'Flow_Lengths']
inp[sections.SUBCATCHMENTS]['S-7']['width'] = gi_parameters_df.at[7,'Original_Widths'] - (sb_7_rg_area+sb_7_gr_area)/gi_parameters_df.at[7,'Flow_Lengths']
inp[sections.SUBCATCHMENTS]['S-8']['width'] = gi_parameters_df.at[8,'Original_Widths'] - (sb_8_rg_area+sb_8_gr_area)/gi_parameters_df.at[8,'Flow_Lengths']
inp[sections.SUBCATCHMENTS]['S-9']['width'] = gi_parameters_df.at[9,'Original_Widths'] - (sb_9_rg_area)/gi_parameters_df.at[9,'Flow_Lengths']


#Set % impevious Removed to Greenspace for each Subbasin
sb_1_imp_reduction = gi_parameters_df.at[1,'Original_Imps'] * (1-(1 - 0))
sb_2_imp_reduction = gi_parameters_df.at[2,'Original_Imps'] * (1-(1 - 0))
sb_3_imp_reduction = gi_parameters_df.at[3,'Original_Imps'] * (1-(1 - 0))
sb_4_imp_reduction = gi_parameters_df.at[4,'Original_Imps'] * (1-(1 - 0))
sb_5_imp_reduction = gi_parameters_df.at[5,'Original_Imps'] * (1-(1 - 0))
sb_6_imp_reduction = gi_parameters_df.at[6,'Original_Imps'] * (1-(1 - 0))
sb_7_imp_reduction = gi_parameters_df.at[7,'Original_Imps'] * (1-(1 - 0))
sb_8_imp_reduction = gi_parameters_df.at[8,'Original_Imps'] * (1-(1 - 0))
sb_9_imp_reduction = gi_parameters_df.at[9,'Original_Imps'] * (1-(1 - 0))

total_imp_area_reduced = sb_1_imp_reduction+sb_2_imp_reduction+sb_3_imp_reduction+sb_4_imp_reduction+sb_5_imp_reduction+sb_6_imp_reduction+sb_7_imp_reduction+sb_8_imp_reduction+sb_9_imp_reduction


#Set New Percent Impervious (Should be same as removed above)
inp[sections.SUBCATCHMENTS]['S-1']['imperviousness'] = inp[sections.SUBCATCHMENTS]['S-1']['imperviousness']*(1-0)
inp[sections.SUBCATCHMENTS]['S-2']['imperviousness'] = inp[sections.SUBCATCHMENTS]['S-2']['imperviousness']*(1-0)
inp[sections.SUBCATCHMENTS]['S-3']['imperviousness'] = inp[sections.SUBCATCHMENTS]['S-3']['imperviousness']*(1-0)
inp[sections.SUBCATCHMENTS]['S-4']['imperviousness'] = inp[sections.SUBCATCHMENTS]['S-4']['imperviousness']*(1-0)
inp[sections.SUBCATCHMENTS]['S-5']['imperviousness'] = inp[sections.SUBCATCHMENTS]['S-5']['imperviousness']*(1-0)
inp[sections.SUBCATCHMENTS]['S-6']['imperviousness'] = inp[sections.SUBCATCHMENTS]['S-6']['imperviousness']*(1-0)
inp[sections.SUBCATCHMENTS]['S-7']['imperviousness'] = inp[sections.SUBCATCHMENTS]['S-7']['imperviousness']*(1-0)
inp[sections.SUBCATCHMENTS]['S-8']['imperviousness'] = inp[sections.SUBCATCHMENTS]['S-8']['imperviousness']*(1-0)
inp[sections.SUBCATCHMENTS]['S-9']['imperviousness'] = inp[sections.SUBCATCHMENTS]['S-9']['imperviousness']*(1-0)






#Update new SWMM INP file with parameters that were set above
inp.write_file(r"MCSWMM-9-GA_Altered_SQ.inp")
#Run New SWMM file
swmm5_run(r"MCSWMM-9-GA_Altered_SQ.inp", swmm_lib_path=r"C:\Program Files (x86)\EPA SWMM 5.1.015\Epaswmm5.exe")
#Create SWMM Results file
out = SwmmOutput(r"MCSWMM-9-GA_Altered_SQ.out")



#_________________ Below is calculation script for nitrogen,phosphorus, and water outputs from each of subbasins into the stream_____


#Get Groundwater Concentrations for each Subbasins Based on Landuse
df_1 = sb_lulc_perc_df.loc[sb_lulc_perc_df['SB'] == 'S-1']
df_1_conc =   (df_1.at['Commercial','Perc']*commercial_n +df_1.at['Residential','Perc']*residential_n + df_1.at['Grasses','Perc']*grasses_n + df_1.at['Roads','Perc']*roads_n +df_1.at['TreeCanopy','Perc']*tree_canopy_n)/1
df_1_conc
df_2 = sb_lulc_perc_df.loc[sb_lulc_perc_df['SB'] == 'S-2']
df_2_conc =   (df_2.at['Commercial','Perc']*commercial_n +df_2.at['Residential','Perc']*residential_n + df_2.at['Grasses','Perc']*grasses_n + df_2.at['Roads','Perc']*roads_n +df_2.at['TreeCanopy','Perc']*tree_canopy_n)/1
df_2_conc
df_3 = sb_lulc_perc_df.loc[sb_lulc_perc_df['SB'] == 'S-3']
df_3_conc =   (df_3.at['Commercial','Perc']*commercial_n +df_3.at['Residential','Perc']*residential_n + df_3.at['Grasses','Perc']*grasses_n + df_3.at['Roads','Perc']*roads_n +df_3.at['TreeCanopy','Perc']*tree_canopy_n)/1
df_3_conc
df_4 = sb_lulc_perc_df.loc[sb_lulc_perc_df['SB'] == 'S-4']
df_4_conc =   (df_4.at['Commercial','Perc']*commercial_n +df_4.at['Residential','Perc']*residential_n + df_4.at['Grasses','Perc']*grasses_n + df_4.at['Roads','Perc']*roads_n +df_4.at['TreeCanopy','Perc']*tree_canopy_n)/1
df_4_conc
df_5 = sb_lulc_perc_df.loc[sb_lulc_perc_df['SB'] == 'S-5']
df_5_conc =   (df_5.at['Commercial','Perc']*commercial_n +df_5.at['Residential','Perc']*residential_n + df_5.at['Grasses','Perc']*grasses_n + df_5.at['Roads','Perc']*roads_n +df_5.at['TreeCanopy','Perc']*tree_canopy_n)/1
df_5_conc
df_6 = sb_lulc_perc_df.loc[sb_lulc_perc_df['SB'] == 'S-6']
df_6_conc =   (df_6.at['Commercial','Perc']*commercial_n +df_6.at['Residential','Perc']*residential_n + df_6.at['Grasses','Perc']*grasses_n + df_6.at['Roads','Perc']*roads_n +df_6.at['TreeCanopy','Perc']*tree_canopy_n)/1
df_6_conc
df_7 = sb_lulc_perc_df.loc[sb_lulc_perc_df['SB'] == 'S-7']
df_7_conc =   (df_7.at['Commercial','Perc']*commercial_n +df_7.at['Residential','Perc']*residential_n + df_7.at['Grasses','Perc']*grasses_n + df_7.at['Roads','Perc']*roads_n +df_7.at['TreeCanopy','Perc']*tree_canopy_n)/1
df_7_conc
df_8 = sb_lulc_perc_df.loc[sb_lulc_perc_df['SB'] == 'S-8']
df_8_conc =   (df_8.at['Commercial','Perc']*commercial_n +df_8.at['Residential','Perc']*residential_n + df_8.at['Grasses','Perc']*grasses_n + df_8.at['Roads','Perc']*roads_n +df_8.at['TreeCanopy','Perc']*tree_canopy_n)/1
df_8_conc
df_9 = sb_lulc_perc_df.loc[sb_lulc_perc_df['SB'] == 'S-9']
df_9_conc =   (df_9.at['Commercial','Perc']*commercial_n +df_9.at['Residential','Perc']*residential_n + df_9.at['Grasses','Perc']*grasses_n + df_9.at['Roads','Perc']*roads_n +df_9.at['TreeCanopy','Perc']*tree_canopy_n)/1
df_9_conc


sb_outs_df = pd.DataFrame()


#Set how much N Ponds reduce subbasin outflow concentrations.
pond_efficiency = 0.15

#Set removal efficiency for each subbasins that has a pond. (Subbasins 5,7,and 8 have ponds) 
sb1_base_tn_conc = df_1_conc
sb2_base_tn_conc = df_2_conc
sb3_base_tn_conc = df_3_conc
sb4_base_tn_conc = df_4_conc
sb5_base_tn_conc = df_5_conc * (1-pond_efficiency)
sb6_base_tn_conc = df_6_conc
sb7_base_tn_conc = df_7_conc * (1- pond_efficiency)
sb8_base_tn_conc = df_8_conc * (1-pond_efficiency)
sb9_base_tn_conc = df_9_conc

#We are assuming a constant TP concentration
all_sb_tp_conc = 0.0025



#Create subbasin 1 dataframes for baseflow and runoff outputs
base_flow_1_out_df = out.get_part('subcatchment','S-1','groundwater_outflow').to_frame()
runoff_flow_1_out_df = out.get_part('subcatchment','S-1','runoff').to_frame()


#Create N Concentration dataframes for subbasin 1 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tn1_out1_df = out.get_part('subcatchment','S-1','TN1').to_frame()
tn2_out1_df = out.get_part('subcatchment','S-1','TN2').to_frame()
tn3_out1_df = out.get_part('subcatchment','S-1','TN3').to_frame()
tn4_out1_df = out.get_part('subcatchment','S-1','TN4').to_frame()
tn5_out1_df = out.get_part('subcatchment','S-1','TN5').to_frame()


#Create P Concentration dataframes for subbasin 1 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tp1_out1_df = out.get_part('subcatchment','S-1','TP1').to_frame()
tp2_out1_df = out.get_part('subcatchment','S-1','TP2').to_frame()
tp3_out1_df = out.get_part('subcatchment','S-1','TP3').to_frame()
tp4_out1_df = out.get_part('subcatchment','S-1','TP4').to_frame()
tp5_out1_df = out.get_part('subcatchment','S-1','TP5').to_frame()


#Script for N,P, and water balance for Subbasin 1
sb_outs_df['SB1_Flow'] = base_flow_1_out_df['subcatchment/S-1/groundwater_outflow'] + runoff_flow_1_out_df['subcatchment/S-1/runoff']
#Set a minimum flowrate from subbasin 1 (cfs)
for i in sb_outs_df.index:
    if sb_outs_df.at[i,'SB1_Flow'] < 2:
        sb_outs_df.at[i,'SB1_Flow'] = 2
sb_outs_df['SB1_Runoff_N_Conc'] =(tn1_out1_df['subcatchment/S-1/TN1'] + tn2_out1_df['subcatchment/S-1/TN2'] + tn3_out1_df['subcatchment/S-1/TN3'] + tn4_out1_df['subcatchment/S-1/TN4'] + tn5_out1_df['subcatchment/S-1/TN5'])
sb_outs_df['SB1_OUT_N_Conc'] = (base_flow_1_out_df['subcatchment/S-1/groundwater_outflow'] * sb1_base_tn_conc + runoff_flow_1_out_df['subcatchment/S-1/runoff'] * sb_outs_df['SB1_Runoff_N_Conc'])/sb_outs_df['SB1_Flow']
sb_outs_df['SB1_Flow'] = base_flow_1_out_df['subcatchment/S-1/groundwater_outflow'] + runoff_flow_1_out_df['subcatchment/S-1/runoff']
sb_outs_df['SB1_Runoff_P_Conc'] =(tp1_out1_df['subcatchment/S-1/TP1'] + tp2_out1_df['subcatchment/S-1/TP2'] + tp3_out1_df['subcatchment/S-1/TP3'] + tp4_out1_df['subcatchment/S-1/TP4'] + tp5_out1_df['subcatchment/S-1/TP5'])
sb_outs_df['SB1_OUT_P_Conc'] = (base_flow_1_out_df['subcatchment/S-1/groundwater_outflow'] * all_sb_tp_conc + runoff_flow_1_out_df['subcatchment/S-1/runoff'] * sb_outs_df['SB1_Runoff_P_Conc'])/sb_outs_df['SB1_Flow']


#Create subbasin 2 dataframes for baseflow and runoff outputs
base_flow_2_out_df = out.get_part('subcatchment','S-2','groundwater_outflow').to_frame()
runoff_flow_2_out_df = out.get_part('subcatchment','S-2','runoff').to_frame()

#Create N Concentration dataframes for subbasin 2 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tn1_out2_df = out.get_part('subcatchment','S-2','TN1').to_frame()
tn2_out2_df = out.get_part('subcatchment','S-2','TN2').to_frame()
tn3_out2_df = out.get_part('subcatchment','S-2','TN3').to_frame()
tn4_out2_df = out.get_part('subcatchment','S-2','TN4').to_frame()
tn5_out2_df = out.get_part('subcatchment','S-2','TN5').to_frame()

#Create P Concentration dataframes for subbasin 2 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tp1_out2_df = out.get_part('subcatchment','S-2','TP1').to_frame()
tp2_out2_df = out.get_part('subcatchment','S-2','TP2').to_frame()
tp3_out2_df = out.get_part('subcatchment','S-2','TP3').to_frame()
tp4_out2_df = out.get_part('subcatchment','S-2','TP4').to_frame()
tp5_out2_df = out.get_part('subcatchment','S-2','TP5').to_frame()


#Script for N,P, and water balance for Subbasin 2
sb_outs_df['SB2_Flow'] = base_flow_2_out_df['subcatchment/S-2/groundwater_outflow'] + runoff_flow_2_out_df['subcatchment/S-2/runoff']
#Set a minimum flowrate from subbasin 2 (cfs)
for i in sb_outs_df.index:
    if sb_outs_df.at[i,'SB2_Flow'] < 2:
        sb_outs_df.at[i,'SB2_Flow'] = 2
sb_outs_df['SB2_Runoff_N_Conc'] =(tn1_out2_df['subcatchment/S-2/TN1'] + tn2_out2_df['subcatchment/S-2/TN2'] + tn3_out2_df['subcatchment/S-2/TN3'] + tn4_out2_df['subcatchment/S-2/TN4'] + tn5_out2_df['subcatchment/S-2/TN5'])
sb_outs_df['SB2_OUT_N_Conc'] = (base_flow_2_out_df['subcatchment/S-2/groundwater_outflow'] * sb2_base_tn_conc + runoff_flow_2_out_df['subcatchment/S-2/runoff'] * sb_outs_df['SB2_Runoff_N_Conc'])/sb_outs_df['SB2_Flow']
sb_outs_df['SB2_Flow'] = base_flow_2_out_df['subcatchment/S-2/groundwater_outflow'] + runoff_flow_2_out_df['subcatchment/S-2/runoff']
sb_outs_df['SB2_Runoff_P_Conc'] =(tp1_out2_df['subcatchment/S-2/TP1'] + tp2_out2_df['subcatchment/S-2/TP2'] + tp3_out2_df['subcatchment/S-2/TP3'] + tp4_out2_df['subcatchment/S-2/TP4'] + tp5_out2_df['subcatchment/S-2/TP5'])
sb_outs_df['SB2_OUT_P_Conc'] = (base_flow_2_out_df['subcatchment/S-2/groundwater_outflow'] * all_sb_tp_conc + runoff_flow_2_out_df['subcatchment/S-2/runoff'] * sb_outs_df['SB2_Runoff_P_Conc'])/sb_outs_df['SB2_Flow']


#Create subbasin 3 dataframes for baseflow and runoff outputs
base_flow_3_out_df = out.get_part('subcatchment','S-3','groundwater_outflow').to_frame()
runoff_flow_3_out_df = out.get_part('subcatchment','S-3','runoff').to_frame()

#Create N Concentration dataframes for subbasin 3 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tn1_out3_df = out.get_part('subcatchment','S-3','TN1').to_frame()
tn2_out3_df = out.get_part('subcatchment','S-3','TN2').to_frame()
tn3_out3_df = out.get_part('subcatchment','S-3','TN3').to_frame()
tn4_out3_df = out.get_part('subcatchment','S-3','TN4').to_frame()
tn5_out3_df = out.get_part('subcatchment','S-3','TN5').to_frame()

#Create P Concentration dataframes for subbasin 3 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tp1_out3_df = out.get_part('subcatchment','S-3','TP1').to_frame()
tp2_out3_df = out.get_part('subcatchment','S-3','TP2').to_frame()
tp3_out3_df = out.get_part('subcatchment','S-3','TP3').to_frame()
tp4_out3_df = out.get_part('subcatchment','S-3','TP4').to_frame()
tp5_out3_df = out.get_part('subcatchment','S-3','TP5').to_frame()


#Script for N,P, and water balance for Subbasin 3
sb_outs_df['SB3_Flow'] = base_flow_3_out_df['subcatchment/S-3/groundwater_outflow'] + runoff_flow_3_out_df['subcatchment/S-3/runoff']
#Set a minimum flowrate from subbasin 3 (cfs)
for i in sb_outs_df.index:
    if sb_outs_df.at[i,'SB3_Flow'] < 1:
        sb_outs_df.at[i,'SB3_Flow'] = 1
sb_outs_df['SB3_Runoff_N_Conc'] =(tn1_out3_df['subcatchment/S-3/TN1'] + tn2_out3_df['subcatchment/S-3/TN2'] + tn3_out3_df['subcatchment/S-3/TN3'] + tn4_out3_df['subcatchment/S-3/TN4'] + tn5_out3_df['subcatchment/S-3/TN5'])
sb_outs_df['SB3_OUT_N_Conc'] = (base_flow_3_out_df['subcatchment/S-3/groundwater_outflow'] * sb3_base_tn_conc + runoff_flow_3_out_df['subcatchment/S-3/runoff'] * sb_outs_df['SB3_Runoff_N_Conc'])/sb_outs_df['SB3_Flow']
sb_outs_df['SB3_Flow'] = base_flow_3_out_df['subcatchment/S-3/groundwater_outflow'] + runoff_flow_3_out_df['subcatchment/S-3/runoff']
sb_outs_df['SB3_Runoff_P_Conc'] =(tp1_out3_df['subcatchment/S-3/TP1'] + tp2_out3_df['subcatchment/S-3/TP2'] + tp3_out3_df['subcatchment/S-3/TP3'] + tp4_out3_df['subcatchment/S-3/TP4'] + tp5_out3_df['subcatchment/S-3/TP5'])
sb_outs_df['SB3_OUT_P_Conc'] = (base_flow_3_out_df['subcatchment/S-3/groundwater_outflow'] * all_sb_tp_conc + runoff_flow_3_out_df['subcatchment/S-3/runoff'] * sb_outs_df['SB3_Runoff_P_Conc'])/sb_outs_df['SB3_Flow']


#Create subbasin 4 dataframes for baseflow and runoff outputs
base_flow_4_out_df = out.get_part('subcatchment','S-4','groundwater_outflow').to_frame()
runoff_flow_4_out_df = out.get_part('subcatchment','S-4','runoff').to_frame()

#Create N Concentration dataframes for subbasin 4 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tn1_out4_df = out.get_part('subcatchment','S-4','TN1').to_frame()
tn2_out4_df = out.get_part('subcatchment','S-4','TN2').to_frame()
tn3_out4_df = out.get_part('subcatchment','S-4','TN3').to_frame()
tn4_out4_df = out.get_part('subcatchment','S-4','TN4').to_frame()
tn5_out4_df = out.get_part('subcatchment','S-4','TN5').to_frame()

#Create P Concentration dataframes for subbasin 4 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tp1_out4_df = out.get_part('subcatchment','S-4','TP1').to_frame()
tp2_out4_df = out.get_part('subcatchment','S-4','TP2').to_frame()
tp3_out4_df = out.get_part('subcatchment','S-4','TP3').to_frame()
tp4_out4_df = out.get_part('subcatchment','S-4','TP4').to_frame()
tp5_out4_df = out.get_part('subcatchment','S-4','TP5').to_frame()


#Script for N,P, and water balance for Subbasin 4
sb_outs_df['SB4_Flow'] = base_flow_4_out_df['subcatchment/S-4/groundwater_outflow'] + runoff_flow_4_out_df['subcatchment/S-4/runoff']
#Set a minimum flowrate from subbasin 4 (cfs)
for i in sb_outs_df.index:
    if sb_outs_df.at[i,'SB4_Flow'] < 1:
        sb_outs_df.at[i,'SB4_Flow'] = 1
sb_outs_df['SB4_Runoff_N_Conc'] =(tn1_out4_df['subcatchment/S-4/TN1'] + tn2_out4_df['subcatchment/S-4/TN2'] + tn3_out4_df['subcatchment/S-4/TN3'] + tn4_out4_df['subcatchment/S-4/TN4'] + tn5_out4_df['subcatchment/S-4/TN5'])
sb_outs_df['SB4_OUT_N_Conc'] = (base_flow_4_out_df['subcatchment/S-4/groundwater_outflow'] * sb4_base_tn_conc + runoff_flow_4_out_df['subcatchment/S-4/runoff'] * sb_outs_df['SB4_Runoff_N_Conc'])/sb_outs_df['SB4_Flow']
sb_outs_df['SB4_Flow'] = base_flow_4_out_df['subcatchment/S-4/groundwater_outflow'] + runoff_flow_4_out_df['subcatchment/S-4/runoff']
sb_outs_df['SB4_Runoff_P_Conc'] =(tp1_out4_df['subcatchment/S-4/TP1'] + tp2_out4_df['subcatchment/S-4/TP2'] + tp3_out4_df['subcatchment/S-4/TP3'] + tp4_out4_df['subcatchment/S-4/TP4'] + tp5_out4_df['subcatchment/S-4/TP5'])
sb_outs_df['SB4_OUT_P_Conc'] = (base_flow_4_out_df['subcatchment/S-4/groundwater_outflow'] * all_sb_tp_conc + runoff_flow_4_out_df['subcatchment/S-4/runoff'] * sb_outs_df['SB4_Runoff_P_Conc'])/sb_outs_df['SB4_Flow']


#Create subbasin 5 dataframes for baseflow and runoff outputs
base_flow_5_out_df = out.get_part('subcatchment','S-5','groundwater_outflow').to_frame()
runoff_flow_5_out_df = out.get_part('subcatchment','S-5','runoff').to_frame()

#Create N Concentration dataframes for subbasin 5 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tn1_out5_df = out.get_part('subcatchment','S-5','TN1').to_frame()
tn2_out5_df = out.get_part('subcatchment','S-5','TN2').to_frame()
tn3_out5_df = out.get_part('subcatchment','S-5','TN3').to_frame()
tn4_out5_df = out.get_part('subcatchment','S-5','TN4').to_frame()
tn5_out5_df = out.get_part('subcatchment','S-5','TN5').to_frame()

#Create P Concentration dataframes for subbasin 5 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tp1_out5_df = out.get_part('subcatchment','S-5','TP1').to_frame()
tp2_out5_df = out.get_part('subcatchment','S-5','TP2').to_frame()
tp3_out5_df = out.get_part('subcatchment','S-5','TP3').to_frame()
tp4_out5_df = out.get_part('subcatchment','S-5','TP4').to_frame()
tp5_out5_df = out.get_part('subcatchment','S-5','TP5').to_frame()


#Script for N,P, and water balance for Subbasin 5
sb_outs_df['SB5_Flow'] = base_flow_5_out_df['subcatchment/S-5/groundwater_outflow'] + runoff_flow_5_out_df['subcatchment/S-5/runoff']
#Set a minimum flowrate from subbasin 5 (cfs)
for i in sb_outs_df.index:
    if sb_outs_df.at[i,'SB5_Flow'] < 1:
        sb_outs_df.at[i,'SB5_Flow'] = 1
sb_outs_df['SB5_Runoff_N_Conc'] =(tn1_out5_df['subcatchment/S-5/TN1'] + tn2_out5_df['subcatchment/S-5/TN2'] + tn3_out5_df['subcatchment/S-5/TN3'] + tn4_out5_df['subcatchment/S-5/TN4'] + tn5_out5_df['subcatchment/S-5/TN5'])
sb_outs_df['SB5_OUT_N_Conc'] = (base_flow_5_out_df['subcatchment/S-5/groundwater_outflow'] * sb5_base_tn_conc + runoff_flow_5_out_df['subcatchment/S-5/runoff'] * sb_outs_df['SB5_Runoff_N_Conc'])/sb_outs_df['SB5_Flow']
sb_outs_df['SB5_Runoff_P_Conc'] =(tp1_out5_df['subcatchment/S-5/TP1'] + tp2_out5_df['subcatchment/S-5/TP2'] + tp3_out5_df['subcatchment/S-5/TP3'] + tp4_out5_df['subcatchment/S-5/TP4'] + tp5_out5_df['subcatchment/S-5/TP5'])
sb_outs_df['SB5_OUT_P_Conc'] = (base_flow_5_out_df['subcatchment/S-5/groundwater_outflow'] * all_sb_tp_conc + runoff_flow_5_out_df['subcatchment/S-5/runoff'] * sb_outs_df['SB5_Runoff_P_Conc'])/sb_outs_df['SB5_Flow']


#Create subbasin 6 dataframes for baseflow and runoff outputs
base_flow_6_out_df = out.get_part('subcatchment','S-6','groundwater_outflow').to_frame()
runoff_flow_6_out_df = out.get_part('subcatchment','S-6','runoff').to_frame()

#Create N Concentration dataframes for subbasin 6 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tn1_out6_df = out.get_part('subcatchment','S-6','TN1').to_frame()
tn2_out6_df = out.get_part('subcatchment','S-6','TN2').to_frame()
tn3_out6_df = out.get_part('subcatchment','S-6','TN3').to_frame()
tn4_out6_df = out.get_part('subcatchment','S-6','TN4').to_frame()
tn5_out6_df = out.get_part('subcatchment','S-6','TN5').to_frame()

#Create P Concentration dataframes for subbasin 6 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tp1_out6_df = out.get_part('subcatchment','S-6','TP1').to_frame()
tp2_out6_df = out.get_part('subcatchment','S-6','TP2').to_frame()
tp3_out6_df = out.get_part('subcatchment','S-6','TP3').to_frame()
tp4_out6_df = out.get_part('subcatchment','S-6','TP4').to_frame()
tp5_out6_df = out.get_part('subcatchment','S-6','TP5').to_frame()


#Script for N,P, and water balance for Subbasin 6
sb_outs_df['SB6_Flow'] = base_flow_6_out_df['subcatchment/S-6/groundwater_outflow'] + runoff_flow_6_out_df['subcatchment/S-6/runoff']
#Set a minimum flowrate from subbasin 6 (cfs)
for i in sb_outs_df.index:
    if sb_outs_df.at[i,'SB6_Flow'] < 1:
        sb_outs_df.at[i,'SB6_Flow'] = 1
sb_outs_df['SB6_Runoff_N_Conc'] =(tn1_out6_df['subcatchment/S-6/TN1'] + tn2_out6_df['subcatchment/S-6/TN2'] + tn3_out6_df['subcatchment/S-6/TN3'] + tn4_out6_df['subcatchment/S-6/TN4'] + tn5_out6_df['subcatchment/S-6/TN5'])
sb_outs_df['SB6_OUT_N_Conc'] = (base_flow_6_out_df['subcatchment/S-6/groundwater_outflow'] * sb6_base_tn_conc + runoff_flow_6_out_df['subcatchment/S-6/runoff'] * sb_outs_df['SB6_Runoff_N_Conc'])/sb_outs_df['SB6_Flow']
sb_outs_df['SB6_Flow'] = base_flow_6_out_df['subcatchment/S-6/groundwater_outflow'] + runoff_flow_6_out_df['subcatchment/S-6/runoff']
sb_outs_df['SB6_Runoff_P_Conc'] =(tp1_out6_df['subcatchment/S-6/TP1'] + tp2_out6_df['subcatchment/S-6/TP2'] + tp3_out6_df['subcatchment/S-6/TP3'] + tp4_out6_df['subcatchment/S-6/TP4'] + tp5_out6_df['subcatchment/S-6/TP5'])
sb_outs_df['SB6_OUT_P_Conc'] = (base_flow_6_out_df['subcatchment/S-6/groundwater_outflow'] * all_sb_tp_conc + runoff_flow_6_out_df['subcatchment/S-6/runoff'] * sb_outs_df['SB6_Runoff_P_Conc'])/sb_outs_df['SB6_Flow']

#Create subbasin 7 dataframes for baseflow and runoff outputs
base_flow_7_out_df = out.get_part('subcatchment','S-7','groundwater_outflow').to_frame()
runoff_flow_7_out_df = out.get_part('subcatchment','S-7','runoff').to_frame()

#Create N Concentration dataframes for subbasin 7 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tn1_out7_df = out.get_part('subcatchment','S-7','TN1').to_frame()
tn2_out7_df = out.get_part('subcatchment','S-7','TN2').to_frame()
tn3_out7_df = out.get_part('subcatchment','S-7','TN3').to_frame()
tn4_out7_df = out.get_part('subcatchment','S-7','TN4').to_frame()
tn5_out7_df = out.get_part('subcatchment','S-7','TN5').to_frame()

#Create P Concentration dataframes for subbasin 7 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tp1_out7_df = out.get_part('subcatchment','S-7','TP1').to_frame()
tp2_out7_df = out.get_part('subcatchment','S-7','TP2').to_frame()
tp3_out7_df = out.get_part('subcatchment','S-7','TP3').to_frame()
tp4_out7_df = out.get_part('subcatchment','S-7','TP4').to_frame()
tp5_out7_df = out.get_part('subcatchment','S-7','TP5').to_frame()


#Script for N,P, and water balance for Subbasin 7
sb_outs_df['SB7_Flow'] = base_flow_7_out_df['subcatchment/S-7/groundwater_outflow'] + runoff_flow_7_out_df['subcatchment/S-7/runoff']
#Set a minimum flowrate from subbasin 7 (cfs)
for i in sb_outs_df.index:
    if sb_outs_df.at[i,'SB7_Flow'] < 2:
        sb_outs_df.at[i,'SB7_Flow'] = 2
sb_outs_df['SB7_Runoff_N_Conc'] =(tn1_out7_df['subcatchment/S-7/TN1'] + tn2_out7_df['subcatchment/S-7/TN2'] + tn3_out7_df['subcatchment/S-7/TN3'] + tn4_out7_df['subcatchment/S-7/TN4'] + tn5_out7_df['subcatchment/S-7/TN5'])
sb_outs_df['SB7_OUT_N_Conc'] = (base_flow_7_out_df['subcatchment/S-7/groundwater_outflow'] * sb7_base_tn_conc + runoff_flow_7_out_df['subcatchment/S-7/runoff'] * sb_outs_df['SB7_Runoff_N_Conc'])/sb_outs_df['SB7_Flow']
sb_outs_df['SB7_Flow'] = base_flow_7_out_df['subcatchment/S-7/groundwater_outflow'] + runoff_flow_7_out_df['subcatchment/S-7/runoff']
sb_outs_df['SB7_Runoff_P_Conc'] =(tp1_out7_df['subcatchment/S-7/TP1'] + tp2_out7_df['subcatchment/S-7/TP2'] + tp3_out7_df['subcatchment/S-7/TP3'] + tp4_out7_df['subcatchment/S-7/TP4'] + tp5_out7_df['subcatchment/S-7/TP5'])
sb_outs_df['SB7_OUT_P_Conc'] = (base_flow_7_out_df['subcatchment/S-7/groundwater_outflow'] * all_sb_tp_conc + runoff_flow_7_out_df['subcatchment/S-7/runoff'] * sb_outs_df['SB7_Runoff_P_Conc'])/sb_outs_df['SB7_Flow']


#Create subbasin 8 dataframes for baseflow and runoff outputs
base_flow_8_out_df = out.get_part('subcatchment','S-8','groundwater_outflow').to_frame()
runoff_flow_8_out_df = out.get_part('subcatchment','S-8','runoff').to_frame()

#Create N Concentration dataframes for subbasin 8 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tn1_out8_df = out.get_part('subcatchment','S-8','TN1').to_frame()
tn2_out8_df = out.get_part('subcatchment','S-8','TN2').to_frame()
tn3_out8_df = out.get_part('subcatchment','S-8','TN3').to_frame()
tn4_out8_df = out.get_part('subcatchment','S-8','TN4').to_frame()
tn5_out8_df = out.get_part('subcatchment','S-8','TN5').to_frame()

#Create P Concentration dataframes for subbasin 8 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tp1_out8_df = out.get_part('subcatchment','S-8','TP1').to_frame()
tp2_out8_df = out.get_part('subcatchment','S-8','TP2').to_frame()
tp3_out8_df = out.get_part('subcatchment','S-8','TP3').to_frame()
tp4_out8_df = out.get_part('subcatchment','S-8','TP4').to_frame()
tp5_out8_df = out.get_part('subcatchment','S-8','TP5').to_frame()


#Script for N,P, and water balance for Subbasin 8
sb_outs_df['SB8_Flow'] = base_flow_8_out_df['subcatchment/S-8/groundwater_outflow'] + runoff_flow_8_out_df['subcatchment/S-8/runoff']
#Set a minimum flowrate from subbasin 8 (cfs)
for i in sb_outs_df.index:
    if sb_outs_df.at[i,'SB8_Flow'] < 2:
        sb_outs_df.at[i,'SB8_Flow'] = 2
sb_outs_df['SB8_Runoff_N_Conc'] =(tn1_out8_df['subcatchment/S-8/TN1'] + tn2_out8_df['subcatchment/S-8/TN2'] + tn3_out8_df['subcatchment/S-8/TN3'] + tn4_out8_df['subcatchment/S-8/TN4'] + tn5_out8_df['subcatchment/S-8/TN5'])
sb_outs_df['SB8_OUT_N_Conc'] = (base_flow_8_out_df['subcatchment/S-8/groundwater_outflow'] * sb8_base_tn_conc + runoff_flow_8_out_df['subcatchment/S-8/runoff'] * sb_outs_df['SB8_Runoff_N_Conc'])/sb_outs_df['SB8_Flow']
sb_outs_df['SB8_Flow'] = base_flow_8_out_df['subcatchment/S-8/groundwater_outflow'] + runoff_flow_8_out_df['subcatchment/S-8/runoff']
sb_outs_df['SB8_Runoff_P_Conc'] =(tp1_out8_df['subcatchment/S-8/TP1'] + tp2_out8_df['subcatchment/S-8/TP2'] + tp3_out8_df['subcatchment/S-8/TP3'] + tp4_out8_df['subcatchment/S-8/TP4'] + tp5_out8_df['subcatchment/S-8/TP5'])
sb_outs_df['SB8_OUT_P_Conc'] = (base_flow_8_out_df['subcatchment/S-8/groundwater_outflow'] * all_sb_tp_conc + runoff_flow_8_out_df['subcatchment/S-8/runoff'] * sb_outs_df['SB8_Runoff_P_Conc'])/sb_outs_df['SB8_Flow']

#Create subbasin 9 dataframes for baseflow and runoff outputs
base_flow_9_out_df = out.get_part('subcatchment','S-9','groundwater_outflow').to_frame()
runoff_flow_9_out_df = out.get_part('subcatchment','S-9','runoff').to_frame()

#Create N Concentration dataframes for subbasin 9 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tn1_out9_df = out.get_part('subcatchment','S-9','TN1').to_frame()
tn2_out9_df = out.get_part('subcatchment','S-9','TN2').to_frame()
tn3_out9_df = out.get_part('subcatchment','S-9','TN3').to_frame()
tn4_out9_df = out.get_part('subcatchment','S-9','TN4').to_frame()
tn5_out9_df = out.get_part('subcatchment','S-9','TN5').to_frame()

#Create P Concentration dataframes for subbasin 9 based on landuse(Note TN1 - TN5 correspond to the 5 different landuses, See Documentation for more details)
tp1_out9_df = out.get_part('subcatchment','S-9','TP1').to_frame()
tp2_out9_df = out.get_part('subcatchment','S-9','TP2').to_frame()
tp3_out9_df = out.get_part('subcatchment','S-9','TP3').to_frame()
tp4_out9_df = out.get_part('subcatchment','S-9','TP4').to_frame()
tp5_out9_df = out.get_part('subcatchment','S-9','TP5').to_frame()


#Subbasin 7 and 8 Share an Outlet into the stream
sb_outs_df['SB7/8_Flow'] = sb_outs_df['SB7_Flow']+sb_outs_df['SB8_Flow']
sb_outs_df['SB7/8_OUT_N_Conc'] = (sb_outs_df['SB7_Flow']*sb_outs_df['SB7_OUT_N_Conc'] + sb_outs_df['SB8_Flow']*sb_outs_df['SB8_OUT_N_Conc'])/sb_outs_df['SB7/8_Flow']
sb_outs_df['SB7/8_OUT_P_Conc'] = (sb_outs_df['SB7_Flow']*sb_outs_df['SB7_OUT_P_Conc'] + sb_outs_df['SB8_Flow']*sb_outs_df['SB8_OUT_P_Conc'])/sb_outs_df['SB7/8_Flow']

#Script for N,P, and water balance for Subbasin 8
sb_outs_df['SB9_Flow'] = base_flow_9_out_df['subcatchment/S-9/groundwater_outflow'] + runoff_flow_9_out_df['subcatchment/S-9/runoff']
#Set a minimum flowrate from subbasin 9 (cfs)
for i in sb_outs_df.index:
    if sb_outs_df.at[i,'SB9_Flow'] < 1:
        sb_outs_df.at[i,'SB9_Flow'] = 1
sb_outs_df['SB9_Runoff_N_Conc'] =(tn1_out9_df['subcatchment/S-9/TN1'] + tn2_out9_df['subcatchment/S-9/TN2'] + tn3_out9_df['subcatchment/S-9/TN3'] + tn4_out9_df['subcatchment/S-9/TN4'] + tn5_out9_df['subcatchment/S-9/TN5'])
sb_outs_df['SB9_OUT_N_Conc'] = (base_flow_9_out_df['subcatchment/S-9/groundwater_outflow'] * sb9_base_tn_conc + runoff_flow_9_out_df['subcatchment/S-9/runoff'] * sb_outs_df['SB9_Runoff_N_Conc'])/sb_outs_df['SB9_Flow']
sb_outs_df['SB9_Flow'] = base_flow_9_out_df['subcatchment/S-9/groundwater_outflow'] + runoff_flow_9_out_df['subcatchment/S-9/runoff']
sb_outs_df['SB9_Runoff_P_Conc'] =(tp1_out9_df['subcatchment/S-9/TP1'] + tp2_out9_df['subcatchment/S-9/TP2'] + tp3_out9_df['subcatchment/S-9/TP3'] + tp4_out9_df['subcatchment/S-9/TP4'] + tp5_out9_df['subcatchment/S-9/TP5'])
sb_outs_df['SB9_OUT_P_Conc'] = (base_flow_9_out_df['subcatchment/S-9/groundwater_outflow'] * all_sb_tp_conc + runoff_flow_9_out_df['subcatchment/S-9/runoff'] * sb_outs_df['SB9_Runoff_P_Conc'])/sb_outs_df['SB9_Flow']


sb_outs_df = sb_outs_df.groupby(np.arange(len(sb_outs_df))//24).mean()
sb_outs_df['Day'] = flow_df['Day']
sb_outs_df = sb_outs_df.reset_index()
    

<font size="6">Code Block 4</font> <br>
<font size="6">Write Stream Ecosystem Functions</font> <br>
-Algal Uptake <br>
-Primary Production <br>
-Mineralization <br>
-Denitrification <br>
-Immobilization <br>
-Miner Uptake <br>

Literature Sources: <br>
(1) Lin, L., Reisinger, A. J., Rosi, E. J., Groffman, P. M., & Band, L. E. (2021). Evaluating Instream Restoration Effectiveness in Reducing Nitrogen Export from an Urban Catchment with a Data-Model Approach. Journal of the American Water Resources Association, 57(3), 449–473. https://doi.org/10.1111/1752-1688.12922 <br>
<br>
(2) Mulholland, P. J., Helton, A. M., Poole, G. C., Hall, R. O., Hamilton, S. K., Peterson, B. J., Tank, J. L., Ashkenas, L. R., Cooper, L. W., Dahm, C. N., Dodds, W. K., Findlay, S. E. G., Gregory, S. V., Grimm, N. B., Johnson, S. L., McDowell, W. H., Meyer, J. L., Valett, H. M., Webster, J. R., … Thomas, S. M. (2008). Stream denitrification across biomes and its response to anthropogenic nitrate loading. Nature, 452(7184), 202–205. https://doi.org/10.1038/nature06686

(3) Webster, J. R., Newbold, J. D., & Lin, L. (2016). Nutrient Spiraling and Transport in Streams: The Importance of In-Stream Biological Processes to Nutrient Dynamics in Streams. The Importance of In-Stream Biological Processes to Nutrient Dynamics in Streams. In Stream Ecosystems in a Changing Environment. https://doi.org/10.1016/B978-0-12-405890-3.00005-1 <br>

(4) Boston, H. L., & Hill, W. R. (1991). Photosynthesis-light relations of stream periphyton communities. 36(4), 644–656 <br>

(5) Stream Solute Workshop. (1990). Concepts and methods for assessing solute dynamics in stream ecosystems. In The North American Benthological Society (Vol. 9, Issue 2)



In [13]:
eco_var_list = []

#creeate stream ecosystem variable dataframe
for i in section_list_UPtoDown:
    eco_var_list.append(str(i)+' '+'NPP (mgC/m2)')
    eco_var_list.append(str(i)+' '+'Algal_N_Uptake (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_P_Uptake (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_C_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_N_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_NC_Ratio')
    eco_var_list.append(str(i)+' '+'Algal_P_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_PC_Ratio')
    eco_var_list.append(str(i)+' '+'Algal_C_Mineralization (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_N_Mineralization (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_P_Mineralization (mg/m2)') 
    eco_var_list.append(str(i)+' '+'Algal_C_Mortality (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_N_Mortality (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_P_Mortality (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_C_Entrained (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_N_Entrained (mg/m2)')
    eco_var_list.append(str(i)+' '+'Algal_P_Entrained (mg/m2)')
    eco_var_list.append(str(i)+' '+'Benthic_C_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Benthic_N_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Benthic_P_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Miner_C_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Miner_N_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Miner_P_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Miner_N_Mineralization (mg/m2)')
    eco_var_list.append(str(i)+' '+'Miner_P_Mineralization (mg/m2)')
    eco_var_list.append(str(i)+' '+'Miner_C_Mortality (mg/m2)')
    eco_var_list.append(str(i)+' '+'Miner_C_Respiration (mg/m2)')
    eco_var_list.append(str(i)+' '+'Immobilizer_C_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Immobilizer_N_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Immobilizer_P_Mass (mg/m2)')
    eco_var_list.append(str(i)+' '+'Immobilizer_N_Mineralization (mg/m2)')
    eco_var_list.append(str(i)+' '+'Immobilizer_P_Mineralization (mg/m2)')
    eco_var_list.append(str(i)+' '+'Immobilizer_C_Mortality (mg/m2)')
    eco_var_list.append(str(i)+' '+'Immobilizer_C_Respiration (mg/m2)')
    eco_var_list.append(str(i)+' '+'Denitrification (mg/m2)')
    eco_var_list.append(str(i)+' '+'Total_N_Mineralization (mg/m2)')
    eco_var_list.append(str(i)+' '+'Total_P_Mineralization (mg/m2)')
    eco_var_list.append(str(i)+' '+'Water_Column_N_Net_Uptake (mg)')
    eco_var_list.append(str(i)+' '+'Water_Column_P_Net_Uptake (mg)')

eco_df = pd.DataFrame(columns = eco_var_list)
eco_df['Day'] = detritus_df['Day']
    


#Algal N Uptake Functions from Webster et al. 2016
def algal_N_Uptake(NO3_col_conc,algal_C_mass):

    ks_n = 0.0015
    umax_n = 0.05300
    k_half_n = 14
    algal_Q10 = 2
    gs = 1/(1+(ks_n*algal_C_mass))
    u_uptake_NO3 = umax_n *(NO3_col_conc/(k_half_n + NO3_col_conc)) * algal_C_mass * gs * algal_Q10f
    return u_uptake_NO3

#Algal Mineralization from Webster et al. 2016
def algal_Mineralization(algal_C_mass, algal_N_mass,algal_P_mass):

    algal_C_mineralization = algal_mineralization_rate*algal_Q10f*algal_C_mass 
    algal_N_mineralization = algal_C_mineralization*(algal_N_mass/algal_C_mass)
    algal_P_mineralization = algal_C_mineralization*(algal_P_mass/algal_C_mass)
    return [algal_C_mineralization,algal_N_mineralization,algal_P_mineralization]


#Algal Mortality Function... Linear function based on standing stock of Algae. Webster et al. 2016
def algal_Mortality(algal_C_mass,algal_N_mass,algal_P_mass):

    algal_C_mortality = algal_Q10f *algal_mortality_rate*algal_C_mass
    algal_N_mortality = algal_C_mortality*(algal_N_mass/algal_C_mass)
    algal_P_mortality = algal_C_mortality*(algal_P_mass/algal_C_mass)
    return [algal_C_mortality,algal_N_mortality,algal_P_mortality]


#Algal Benthic Entrainment from Lin et al. 2021
def algal_Entrainment(velocity,algal_C_mass,algal_N_mass,algal_P_mass):

    nc_ratio = algal_N_mass/algal_C_mass
    pc_ratio = algal_P_mass/algal_C_mass
    algal_C_entrainment_frac = (0.85/(1+math.e**-((velocity-0.5)/0.06)))
    algal_C_entrainment = algal_C_entrainment_frac*algal_C_mass
    algal_N_entrainment = algal_C_entrainment * nc_ratio
    algal_P_entrainment = algal_C_entrainment * pc_ratio
    return [algal_C_entrainment,algal_N_entrainment,algal_P_entrainment]



#Algal P Uptake from Webster et al. 2016
def algal_P_Uptake(PO4_col_conc,algal_C_mass):
    
    ks = 0.0015
    umax_p = 0.00731*0.2 # (Value was reduced to 20% during calibration process. Model would not run at original value)
    k_half_p = 2
    algal_Q10 = 2
    gs = 1/(1+(ks*algal_C_mass))
    u_uptake_PO4 = umax_p *(PO4_col_conc/(k_half_p + PO4_col_conc)) * algal_C_mass * gs * algal_Q10f
    return u_uptake_PO4


#Calculate Primary Production. Functions from Webster et al. 2016 and Boston & Hill 1991
def NPP_Calculator(light_df,algal_NC_ratio,algal_PC_ratio,algal_carbon_mass,zone):

    
    ks_n = 0.0015
    #Subsistence N:C and P:C Ratios for algae
    n_c_quota = 0.0606
    p_c_quota = 0.00377


# PAR values all came from a sensor at section 3 (zone 7).. Photo_C_Maxes were calibrated to correspond to each zone. 
# If PAR data is available at all reach sections over the simulation period than Photo C Max does not have to be a calibrated parameter
    if zone == 7:
        initial_slope_LRF = 0.8
        photo_C_max = 0.00095
    elif zone == 'K':
        initial_slope_LRF = 0.8
        photo_C_max = 0.0008
    elif zone == 4:
        initial_slope_LRF = 0.8
        photo_C_max = 0.00075

    for i in light_df.index:
# If PAR data is available at all reach sections over the simulation period than line directly below should be:light_df.at[i,'LRF'] = math.tanh(initial_slope_LRF*light_df.at[i,f'PAR_{zone}']/photo_C_max)
        light_df.at[i,'LRF'] = math.tanh(initial_slope_LRF*light_df.at[i,f'PAR_{7}']/photo_C_max)
        light_df.at[i,'Potential_Photo_C'] = photo_C_max * light_df.at[i,'LRF']*light_df.at[i,f'PAR_{7}']

    gL = light_df['Potential_Photo_C'].sum()
    gs = 1/(1+(ks_n*algal_carbon_mass))
    g1 = min((1-(n_c_quota/algal_NC_ratio)),(1-(p_c_quota/algal_PC_ratio)))
    npp = algal_carbon_mass*gL*gs*abs(g1)*algal_Q10f
    return npp 


#Benthic Detritus Entrainment and Deposition function from Lin et al. 2016
def calculate_benthic_detritus_flux(day,benthic_area,velocity,volume,benthic_C,benthic_N,benthic_P,water_column_detritus_C,water_column_detritus_N,water_column_detritus_P):

    deposit_frac = 1-math.e**(-30*benthic_area*0.05/(velocity*volume))

    if deposit_frac >= 0.9:
        deposit_frac = 0.9

    deposit_C = deposit_frac * water_column_detritus_C
    deposit_N = deposit_frac * water_column_detritus_N
    deposit_P = deposit_frac * water_column_detritus_P
    entrain_frac = (0.85/(1+math.e**-((velocity-0.4)/0.06)))

    if entrain_frac >= 1:
        entrain_frac = 0.99

    entrain_C = entrain_frac * benthic_C
    entrain_N = entrain_frac * benthic_N
    entrain_P = entrain_frac * benthic_P
    net_detritusC_input = deposit_C - entrain_C
    net_detritusN_input = deposit_N - entrain_N
    net_detritusP_input = deposit_P - entrain_P
    return [net_detritusC_input,net_detritusN_input,net_detritusP_input,deposit_C, deposit_N,deposit_P,entrain_C,entrain_N,entrain_P]

#Set Miner Preliminary variables from Webster et al. 2016
miner_C_N_molar_quota = 15
miner_C_P_molar_quota = 1000
miner_C_N_mass_quota = 12.8
miner_C_P_mass_quota = 387.7
kmax_m = 0.01
detritus_Q10 = 2
miner_detritus_frac = 0.0
initial_miner_C = np.nan
iniital_miner_N = initial_miner_C/miner_C_N_molar_quota
iniital_miner_p = initial_miner_C/miner_C_P_molar_quota
miner_respiration_rate = 0.05
miner_C = np.nan
miner_N = np.nan
miner_P = np.nan
velocity = np.nan


#Miner Uptake process functions from Webster et al. 2016
def calculate_miner_processes(benthic_C,benthic_N,benthic_P,miner_C_mass,miner_N,miner_P):
    
    gm = 0.01

    if benthic_C/benthic_N > miner_CN_mass_quota or benthic_C/benthic_P > miner_CP_mass_quota:
        g_m_m_c = gm*benthic_C*detritus_Q10f
        g_m_m_n = g_m_m_c*(benthic_N/benthic_C)
        g_m_m_p = g_m_m_c*(benthic_P/benthic_C)

    else:
        g_m_m_c = min(benthic_N * miner_CN_mass_quota*gm*detritus_Q10f, benthic_P*miner_CP_mass_quota*gm*detritus_Q10f)
        g_m_m_n = g_m_m_c/miner_CN_mass_quota
        g_m_m_p = g_m_m_c/miner_CP_mass_quota

        
    miner_death_C = 0.02*miner_C_mass
    miner_C_respiration = miner_respiration_rate*miner_C_mass*detritus_Q10f
    return [g_m_m_c,g_m_m_n,g_m_m_p,miner_death_C,miner_C_respiration]


#Set Immobilizer Preliminary variables from Webster et al. 2016
immobilizer_C_N_molar_quota = 5
immobilizer_C_P_molar_quota = 100
immobilizer_C_N_mass_quota = 4.3
immobilizer_C_P_mass_quota = 38.8
kmax_i = 0.01
khalf_n_i = 90
khalf_p_i = 20
detritus_Q10 = 2
initial_immobilizer_C = np.nan
iniital_immobilizer_N = initial_immobilizer_C/immobilizer_C_N_mass_quota
iniital_immobilizer_p = initial_immobilizer_C/immobilizer_C_P_mass_quota
immobilizer_respiration_rate = 0.2
immobilizer_C = np.nan

#Immobilizer Uptake Processes from Webster et al. 2016
def calculate_immobilizer_processes(benthic_C,input_tn,input_tp,immobilizer_C):

    gi = 0.1
    n_conc = input_tn
    p_conc = input_tp
    g_m_i = gi*benthic_C*detritus_Q10f*min((n_conc/(khalf_n_i+n_conc)),(p_conc/(khalf_p_i+p_conc)))
    if g_m_i < 0:
        g_m_i = 0
    immobilizer_C_death = 0.1*immobilizer_C
    immobilizer_C_respiration = immobilizer_respiration_rate*immobilizer_C*detritus_Q10f
    return [g_m_i, immobilizer_C_death,immobilizer_C_respiration]


#Denitrification from Mullhollan et al 2008 and Stream Solute Workshop 1990
def denitrification(no3_conc,benthic_area,flow):
    
    vf_den = 10**(-2.975 -0.493* math.log10(0.96*no3_conc))
    vf_den = vf_den*0.01
    den = vf_den*0.96*no3_conc*86400
    return den


#Function that combines all the above functions which is used in the stream ecosystem Model
def SSHBS_Uptake_Fxns(day,input_TN_conc,input_TP_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow):

    


            algal_mineralization  = algal_Mineralization(algal_C_mass,algal_N_mass, algal_P_mass)
            algal_C_mineralization = algal_mineralization[0]
            algal_N_mineralization = algal_mineralization[1]
            algal_P_mineralization = algal_mineralization[2]
            algal_mortality = algal_Mortality(algal_C_mass,algal_N_mass,algal_P_mass)
            algal_C_mortality = algal_mortality[0]
            algal_N_mortality = algal_mortality[1]
            algal_P_mortality = algal_mortality[2]
            algal_entrainment = algal_Entrainment(velocity,algal_C_mass,algal_N_mass,algal_P_mass)
            algal_C_entrained =  algal_entrainment[0] 
            algal_N_entrained = algal_entrainment[1] 
            algal_P_entrained = algal_entrainment[2]
            npp = NPP_Calculator(light_df,algal_NC_ratio,algal_PC_ratio,algal_C_mass,zone)

            new_algal_C_mass = algal_C_mass+ npp - algal_C_mineralization - algal_C_mortality - algal_C_entrained
            if new_algal_C_mass < 0:
                new_algal_C_mass = 10

            algal_N_uptake = algal_N_Uptake(input_TN_conc,new_algal_C_mass)
            algal_P_uptake = algal_P_Uptake(input_TP_conc,new_algal_C_mass)

        
            if algal_N_uptake < 0:
                algal_N_uptake = 0
            if algal_P_uptake < 0:
                algal_P_uptake = 0




            benthic_entrainment_frac = (0.85/(1+math.e**-((velocity-0.4)/0.06)))
            deposit_frac = 1-math.e**(-30*benthic_area*0.05/(velocity*volume))

            miner_C_mass = miner_C_mass - miner_C_mass*benthic_entrainment_frac + water_column_miner_C*deposit_frac
            miner_N_mass = miner_C_mass/miner_CN_mass_quota 
            miner_P_mass = miner_C_mass/miner_CP_mass_quota 
            entrain_miner_C_mass = miner_C_mass*benthic_entrainment_frac
            entrain_miner_N_mass = (miner_C_mass/miner_CN_mass_quota)*benthic_entrainment_frac
            entrain_miner_P_mass = (miner_C_mass/miner_CP_mass_quota)*benthic_entrainment_frac
            deposit_miner_C_mass =  water_column_miner_C*deposit_frac
            deposit_miner_N_mass =  water_column_miner_N*deposit_frac
            deposit_miner_P_mass =  water_column_miner_P*deposit_frac
            immobilizer_C_mass = immobilizer_C_mass - immobilizer_C_mass*benthic_entrainment_frac + water_column_immobilizer_C*deposit_frac
            immobilizer_N_mass = immobilizer_N_mass - immobilizer_N_mass*benthic_entrainment_frac + water_column_immobilizer_N*deposit_frac
            immobilizer_P_mass = immobilizer_P_mass - immobilizer_P_mass*benthic_entrainment_frac + water_column_immobilizer_P*deposit_frac
            entrain_immobilizer_C_mass = immobilizer_C_mass*benthic_entrainment_frac
            entrain_immobilizer_N_mass = (immobilizer_C_mass/immobilizer_CN_mass_quota)*benthic_entrainment_frac
            entrain_immobilizer_P_mass = (immobilizer_C_mass/miner_CP_mass_quota)*benthic_entrainment_frac
            deposit_immobilizer_C_mass =  water_column_immobilizer_C*deposit_frac
            deposit_immobilizer_N_mass =  water_column_immobilizer_N*deposit_frac
            deposit_immobilizer_P_mass =  water_column_immobilizer_P*deposit_frac




            benthic_detritus_flux = calculate_benthic_detritus_flux(day,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,detritus_C,detritus_N,detritus_P)
            

            new_benthic_C_mass = benthic_C_mass + benthic_detritus_flux[0] + algal_C_mortality
            new_benthic_N_mass = benthic_N_mass + benthic_detritus_flux[1] + algal_N_mortality
            new_benthic_P_mass = benthic_P_mass + benthic_detritus_flux[2] + algal_P_mortality
            deposit_detritus_C_mass = benthic_detritus_flux[3]
            deposit_detritus_N_mass = benthic_detritus_flux[4]
            deposit_detritus_P_mass = benthic_detritus_flux[5]
            entrain_detritus_C_mass = benthic_detritus_flux[6]
            entrain_detritus_N_mass = benthic_detritus_flux[7] 
            entrain_detritus_P_mass = benthic_detritus_flux[8]

            if new_benthic_C_mass <= 0:
                new_benthic_C_mass = 100
                new_benthic_N_mass = new_benthic_C_mass/59
                new_benthic_P_mass = new_benthic_C_mass/3204


            miner_processes = calculate_miner_processes(new_benthic_C_mass,new_benthic_N_mass,new_benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass)
            miner_C_assimilation = miner_processes[0]
            miner_N_assimilation = miner_processes[1]
            miner_P_assimilation = miner_processes[2]
            miner_C_mortality = miner_processes[3]
            miner_N_mortality = miner_C_mortality/miner_CN_mass_quota
            miner_P_mortality = miner_C_mortality/miner_CP_mass_quota
            miner_C_respiration = miner_processes[4]
            miner_N_respiration =  miner_C_respiration/miner_CN_mass_quota
            miner_P_respiration =  miner_C_respiration/miner_CP_mass_quota

            new_miner_C_mass = miner_C_mass + miner_C_assimilation - miner_C_mortality - miner_C_respiration
            new_miner_N_mass = miner_N_mass + miner_N_assimilation - miner_N_mortality - miner_N_respiration
            new_miner_P_mass = miner_P_mass + miner_P_assimilation - miner_P_mortality - miner_P_respiration

            miner_C_mineralization = 0
            miner_N_mineralization = 0
            miner_P_mineralization = 0
            if new_miner_C_mass/new_miner_N_mass > miner_CN_mass_quota:
                miner_C_mineralization = new_miner_C_mass - miner_CN_mass_quota*new_miner_N_mass
                new_miner_C_mass = new_miner_C_mass - miner_C_mineralization


            elif new_miner_C_mass/new_miner_P_mass > miner_CP_mass_quota:
                miner_C_mineralization = new_miner_C_mass - miner_CP_mass_quota*new_miner_P_mass
                new_miner_C_mass = new_miner_C_mass - miner_C_mineralization



            miner_N_mineralization =  miner_N_respiration
            miner_P_mineralization =  miner_P_respiration

            new_miner_N_mass = new_miner_N_mass - miner_N_mineralization
            new_miner_P_mass = new_miner_P_mass - miner_P_mineralization


            immobilizer_processes = calculate_immobilizer_processes(new_benthic_C_mass,input_TN_conc,input_TP_conc,immobilizer_C_mass)
            immobilizer_C_assimilation = immobilizer_processes[0]
            immobilizer_N_assimilation = immobilizer_C_assimilation/immobilizer_CN_mass_quota
            immobilizer_P_assimilation = immobilizer_C_assimilation/immobilizer_CP_mass_quota
            immobilizer_C_mortality = immobilizer_processes[1]
            immobilizer_N_mortality = immobilizer_C_mortality/immobilizer_CN_mass_quota
            immobilizer_P_mortality = immobilizer_C_mortality/immobilizer_CP_mass_quota
            immobilizer_C_respiration = immobilizer_processes[2]
            immobilizer_N_respiration = immobilizer_C_respiration/immobilizer_CN_mass_quota
            immobilizer_P_respiration = immobilizer_C_respiration/immobilizer_CP_mass_quota
            immobilizer_N_mineralization  = immobilizer_N_respiration 
            immobilizer_P_mineralization  = immobilizer_P_respiration 
            new_immobilizer_C_mass = immobilizer_C_mass + immobilizer_C_assimilation - immobilizer_C_mortality - immobilizer_C_respiration
            new_immobilizer_N_mass = immobilizer_N_mass + immobilizer_N_assimilation - immobilizer_N_mineralization
            new_immobilizer_P_mass = immobilizer_P_mass + immobilizer_P_assimilation - immobilizer_P_mineralization
            new_benthic_C_mass = new_benthic_C_mass + immobilizer_C_mortality + miner_C_mortality
            new_benthic_N_mass = new_benthic_N_mass + immobilizer_N_mortality + miner_N_mortality
            new_benthic_P_mass = new_benthic_P_mass + immobilizer_P_mortality + miner_P_mortality

            if new_immobilizer_C_mass < 0:
                j = 0 - new_immobilizer_C_mass
                increase_ratio = j/new_immobilizer_C_mass
                immobilizer_C_mortality = immobilizer_C_mortality - immobilizer_C_mortality*increase_ratio
                immobilizer_C_respiration = immobilizer_C_respiration - immobilizer_C_respiration*increase_ratio
                immobilizer_N_mineralization = immobilizer_N_mineralization - immobilizer_N_mineralization*increase_ratio
                immobilizer_P_mineralization = immobilizer_P_mineralization - immobilizer_P_mineralization*increase_ratio
                new_immobilizer_C_mass = immobilizer_C_mass + immobilizer_C_assimilation - immobilizer_C_mortality - immobilizer_C_respiration
                new_immobilizer_N_mass = immobilizer_N_mass + immobilizer_N_assimilation - immobilizer_N_mineralization
                new_immobilizer_P_mass = immobilizer_P_mass + immobilizer_P_assimilation - immobilizer_P_mineralization




            if new_immobilizer_N_mass < 0:
                j = 0 - new_immobilizer_N_mass
                increase_ratio = j/new_immobilizer_N_mass
                immobilizer_C_mortality = immobilizer_C_mortality - immobilizer_C_mortality*increase_ratio
                immobilizer_C_respiration = immobilizer_C_respiration - immobilizer_C_respiration*increase_ratio
                immobilizer_N_mineralization = immobilizer_N_mineralization - immobilizer_N_mineralization*increase_ratio
                immobilizer_P_mineralization = immobilizer_P_mineralization - immobilizer_P_mineralization*increase_ratio
                new_immobilizer_C_mass = immobilizer_C_mass + immobilizer_C_assimilation - immobilizer_C_mortality - immobilizer_C_respiration
                new_immobilizer_N_mass = immobilizer_N_mass + immobilizer_N_assimilation - immobilizer_N_mineralization
                new_immobilizer_P_mass = immobilizer_P_mass + immobilizer_P_assimilation - immobilizer_P_mineralization

            if new_immobilizer_P_mass < 0:
                j = 0 - new_immobilizer_P_mass
                increase_ratio = j/new_immobilizer_N_mass
                immobilizer_C_mortality = immobilizer_C_mortality - immobilizer_C_mortality*increase_ratio
                immobilizer_C_respiration = immobilizer_C_respiration - immobilizer_C_respiration*increase_ratio
                immobilizer_N_mineralization = immobilizer_N_mineralization - immobilizer_N_mineralization*increase_ratio
                immobilizer_P_mineralization = immobilizer_P_mineralization - immobilizer_P_mineralization*increase_ratio
                new_immobilizer_C_mass = immobilizer_C_mass + immobilizer_C_assimilation - immobilizer_C_mortality - immobilizer_C_respiration
                new_immobilizer_N_mass = immobilizer_N_mass + immobilizer_N_assimilation - immobilizer_N_mineralization
                new_immobilizer_P_mass = immobilizer_P_mass + immobilizer_P_assimilation - immobilizer_P_mineralization



            den = denitrification(input_TN_conc,benthic_area,flow)

            detritus_C_respiration = new_benthic_C_mass*detritus_Q10f*0.2 #Calibrated detritus respiration... Reduced to 20% of literature value
            detritus_N_respiration = detritus_C_respiration*(benthic_N_mass/benthic_C_mass)
            detritus_P_respiration = detritus_C_respiration*(benthic_P_mass/benthic_C_mass)

            total_N_mineralization = algal_N_mineralization + miner_N_mineralization + immobilizer_N_mineralization + detritus_N_respiration
            total_P_mineralization = algal_P_mineralization + miner_P_mineralization + immobilizer_P_mineralization + detritus_P_respiration


            water_column_N_net_uptake = (algal_N_uptake + immobilizer_N_assimilation + den - total_N_mineralization - detritus_N_respiration)*benthic_area

            if water_column_N_net_uptake > (input_TN_conc*volume):
                j = 0.99*(input_TN_conc*volume)
                reduction_ratio = j/water_column_N_net_uptake
                water_column_N_net_uptake = j
                algal_N_uptake = reduction_ratio*algal_N_uptake
                immobilizer_N_assimilation = reduction_ratio*immobilizer_N_assimilation
                immobilizer_C_assimilation = immobilizer_N_assimilation*immobilizer_CN_mass_quota
                immobilizer_P_assimilation = immobilizer_C_assimilation/immobilizer_CP_mass_quota
                den = reduction_ratio*den


            water_column_P_net_uptake = (algal_P_uptake + immobilizer_P_assimilation - total_P_mineralization - detritus_P_respiration)*benthic_area

            if water_column_P_net_uptake > (input_TP_conc*volume):
                j = 0.99*(input_TP_conc*volume)
                reduction_ratio = j/water_column_P_net_uptake
                water_column_P_net_uptake = j
                algal_P_uptake = reduction_ratio*algal_P_uptake
                immobilizer_P_assimilation = reduction_ratio*immobilizer_P_assimilation
                immobilizer_C_assimilation = immobilizer_P_assimilation*immobilizer_CP_mass_quota
                immobilizer_N_assimilation = immobilizer_C_assimilation/immobilizer_CN_mass_quota

                new_immobilizer_C_mass = immobilizer_C_mass + immobilizer_C_assimilation - immobilizer_C_mortality - immobilizer_C_respiration
                new_immobilizer_C_mass = immobilizer_C_mass + immobilizer_C_assimilation - immobilizer_C_mortality - immobilizer_C_respiration
                new_immobilizer_N_mass = immobilizer_N_mass + immobilizer_N_assimilation - immobilizer_N_mineralization
                new_immobilizer_P_mass = immobilizer_P_mass + immobilizer_P_assimilation - immobilizer_P_mineralization



        
            new_benthic_C_mass = new_benthic_C_mass - miner_C_assimilation - immobilizer_C_assimilation - detritus_C_respiration
            new_benthic_N_mass = new_benthic_N_mass - miner_N_assimilation - detritus_N_respiration
            new_benthic_P_mass = new_benthic_P_mass - miner_P_assimilation - detritus_P_respiration

            
            new_algal_N_mass = algal_N_mass + algal_N_uptake - algal_N_mineralization-algal_N_mortality - algal_N_entrained
            new_algal_P_mass = algal_P_mass + algal_P_uptake-algal_P_mineralization-algal_P_mortality - algal_P_entrained
            
            new_algal_NC_ratio = new_algal_N_mass/new_algal_C_mass
            new_algal_PC_ratio = new_algal_P_mass/new_algal_C_mass
                
            




            return  [npp,algal_N_uptake,algal_P_uptake,new_algal_C_mass,new_algal_N_mass,new_algal_NC_ratio,new_algal_P_mass,new_algal_PC_ratio,algal_C_mineralization,algal_N_mineralization,algal_P_mineralization,algal_C_mortality,algal_N_mortality,algal_P_mortality,algal_C_entrained,algal_N_entrained,algal_P_entrained,new_benthic_C_mass,new_benthic_N_mass,new_benthic_P_mass,new_miner_C_mass,new_miner_N_mass,new_miner_P_mass,miner_N_mineralization,miner_P_mineralization,miner_C_mortality,miner_C_respiration,new_immobilizer_C_mass,new_immobilizer_N_mass,new_immobilizer_P_mass,immobilizer_N_mineralization,immobilizer_P_mineralization,immobilizer_C_mortality, immobilizer_C_respiration,den,total_N_mineralization,total_P_mineralization,water_column_N_net_uptake,water_column_P_net_uptake,detritus_C_respiration, entrain_detritus_C_mass, entrain_detritus_N_mass, entrain_detritus_P_mass,entrain_miner_C_mass,entrain_miner_N_mass,entrain_miner_P_mass,entrain_immobilizer_C_mass,entrain_immobilizer_N_mass,entrain_immobilizer_P_mass,deposit_detritus_C_mass,deposit_detritus_N_mass,deposit_detritus_P_mass,deposit_miner_C_mass,deposit_miner_N_mass,deposit_miner_P_mass,deposit_immobilizer_C_mass,deposit_immobilizer_N_mass,deposit_immobilizer_P_mass,miner_N_assimilation,immobilizer_N_assimilation]


#Additional Stream Ecostystem Parameterization
algal_NC_ratio = 0.06
algal_PC_ratio = 0.0032
initial_algal_C_mass = 500


algal_mineralization_rate = 0.08
algal_Q10 = 1.1
algal_mortality_rate = 0.005


max_NC_ratio = (1/10.4)
max_PC_ratio = (1/123.817)

initial_benthic_C = 752
initial_benthic_N = 12.75
initial_benthic_P = 0.2

miner_CN_molar_quota = 15
miner_CP_molar_quota = 1000

miner_CN_mass_quota = 12.8
miner_CP_mass_quota = 387.7

miner_NP_mass_quota = (12.8/387.7)

kmax_m = 0.1
initial_miner_C = 100
initial_miner_N = initial_miner_C/miner_CN_mass_quota
initial_miner_P = initial_miner_C/miner_CP_mass_quota



immobilizer_CN_molar_quota = 5
immobilizer_CP_molar_quota = 100
immobilizer_CN_mass_quota = 4.3
immobilizer_CP_mass_quota = 38.8
kmax_i = 1
khalf_n_i = 90
khalf_p_i = 20
detritus_Q10 = 1.3
initial_immobilizer_C = 1
initial_immobilizer_N = initial_immobilizer_C/immobilizer_CN_mass_quota
initial_immobilizer_P = initial_immobilizer_C/immobilizer_CP_mass_quota


<font size="6">Code Block 5</font> <br>
<font size="6">Execution of Channel Hydraulic and In-Stream Ecostystem Model</font> <br>

In [14]:

#______________________SB1_SB2_Inflow___________

#Set zone
zone = 4
#Group List of HEC-RAS cross-sections in the group
group_list = [3382, 3352, 3322, 3292, 3262, 3232, 3202, 3172, 3142, 3112, 3083, 3052, 3022]

#Model runs iteratively over each day
for day in flow_df.index:
    #Model computes flow hydraulics and ecosystem functions iteratively over each HEC-RAS cross-section in the group_list
    for section in group_list:

        #Nutrients N and P and flow from 1st subbasin converted from mg/L to mg/m3 and cfs to cms respectively
        df1_tn_C = sb_outs_df.at[day,'SB1_OUT_N_Conc']*1000
        df1_tp_C = sb_outs_df.at[day,'SB1_OUT_P_Conc']*1000
        df1_Q =  sb_outs_df.at[day,'SB1_Flow']/35.315

        #Nutrients N and P and flow from 2nd subbasin converted from mg/L to mg/m3 and cfs to cms respectively
        df2_tn_C = sb_outs_df.at[day,'SB2_OUT_N_Conc']*1000
        df2_tp_C = sb_outs_df.at[day,'SB2_OUT_P_Conc']*1000
        df2_Q =  sb_outs_df.at[day,'SB2_Flow']/35.315
        
        #Combine flows from subbasin 1 and subbasin 2
        flow_df.at[day,f"{section} Flow (cms)"] = df1_Q + df2_Q

        #Mass balance calculations for N and P from subbasin 1 and 2
        dfout_tn_C = (df1_tn_C*df1_Q +df2_tn_C*df2_Q)/(df1_Q + df2_Q)
        dfout_tp_C = (df1_tp_C*df1_Q +df2_tp_C*df2_Q)/(df1_Q + df2_Q)

        
        #Start at first HEC-RAS cross section in the group list
        if section == 3382:

            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, cross-sectional area, and approximated benthic area.
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            flow_df.at[day,f'{3382} SWMM_TN_Conc (mg/m3)'] = dfout_tn_C
            flow_df.at[day,f'{3382} SWMM_TP_Conc (mg/m3)'] = dfout_tp_C
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{section} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = dfout_tn_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = dfout_tp_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = dfout_tn_C
            tp_conc = dfout_tp_C
        
        #Apply flow hydraulics interpolation to the rest of HEC-RAS cross-sections in group list
        else:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, and approximated benthic area
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{section} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Conc (mg/m3)'] = upstream_tn_conc
            flow_df.at[day,f'{section} SWMM_TP_Conc (mg/m3)'] = upstream_tp_conc
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = upstream_tn_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = upstream_tp_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = upstream_tn_conc
            tp_conc = upstream_tp_conc
        
        #Set water column detritus and microbes to zero at the most upstream part of the group
        if section == 3382:
            water_column_detritus_C = 0
            water_column_detritus_N = 0
            water_column_detritus_P = 0
            water_column_miner_C = 0
            water_column_miner_N = 0
            water_column_miner_P = 0
            water_column_immobilizer_C = 0
            water_column_immobilizer_N = 0
            water_column_immobilizer_P = 0

        
            #Ecosystem Function Initilization for day zero
            if day == 0:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus deposition value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus deposition value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']

            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]            
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]


            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']

            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']

                
    # Apply Ecosystem Uptake Equations HERE for TN and TP! These update concentrations. ('else' implies this is now for the rest of the cross-sections in the group)
    #Some values depend on boundary conditions at upstream cross-sections here.
        else:
            #Ecosystem Function Initilization for day zero
            if day == 0:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus deposition value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero (same for all cross-section, these are set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (Same for all cross-sections, these is set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus deposition value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set the daily flow hydraulics
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']

            #Execute all stream ecosystem functions using initialized values 
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]
            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']

            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']



#______________________SB4_Inflow___________

#New Group_List since a tributary's flow changes stream flow and nutrient mass balance at this part of the stream

zone = 4
#Group List of HEC-RAS cross-sections in the group
group_list = [2992,2962,2932,2900,2842,2812]


#Model runs iteratively over each day
for day in flow_df.index:
    #Model computes flow hydraulics and ecosystem functions iteratively over each HEC-RAS cross-section in the group_list
    for section in group_list:

        #Nutrients N and P and flow from 1st subbasin (SB4 in this case) converted from mg/L to mg/m3 and cfs to cms respectively
        df1_tn_C = sb_outs_df.at[day,'SB4_OUT_N_Conc']*1000
        df1_tp_C = sb_outs_df.at[day,'SB4_OUT_P_Conc']*1000
        df1_Q =  sb_outs_df.at[day,'SB4_Flow']/35.315

        #Nutrients N and P and flow from UPSTREAM converted from mg/L to mg/m3 and cfs to cms respectively
        df2_tn_C = flow_df.at[day,f'{3022} Updated_TN_Conc (mg/m3)']
        df2_tp_C = flow_df.at[day,f'{3022} Updated_TP_Conc (mg/m3)']
        df2_Q = flow_df.at[day,f"{3022} Flow (cms)"]
        # flow_df.at[day,'2812 Flow (cms)']
        
        #Combine flows from subbasin tributary and UPSTREAM
        flow_df.at[day,f"{section} Flow (cms)"] = df1_Q + df2_Q

        #Mass balance calculations for N and P from new subbasin tributary and UPSTREAM
        dfout_tn_C = (df1_tn_C*df1_Q +df2_tn_C*df2_Q)/(df1_Q + df2_Q)
        dfout_tp_C = (df1_tp_C*df1_Q +df2_tp_C*df2_Q)/(df1_Q + df2_Q)


     #Start at first HEC-RAS cross section in the group list
        if section == 2992:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, cross-sectional area, and approximated benthic area.
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            flow_df.at[day,f'{2992} SWMM_TN_Conc (mg/m3)'] = dfout_tn_C
            flow_df.at[day,f'{2992} SWMM_TP_Conc (mg/m3)'] = dfout_tp_C
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{section} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = dfout_tn_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = dfout_tp_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = dfout_tn_C
            tp_conc = dfout_tp_C

     #Apply flow hydraulics interpolation to the rest of HEC-RAS cross-sections in group list
        else:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, and approximated benthic area
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{section} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Conc (mg/m3)'] = upstream_tn_conc
            flow_df.at[day,f'{section} SWMM_TP_Conc (mg/m3)'] = upstream_tp_conc
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = upstream_tn_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = upstream_tp_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = upstream_tn_conc
            tp_conc = upstream_tp_conc
        
        #Set water column detritus and microbes to zero at the most upstream part of the group.
        if section == 2992:
            water_column_detritus_C = 0
            water_column_detritus_N = 0
            water_column_detritus_P = 0
            water_column_miner_C = 0
            water_column_miner_N = 0
            water_column_miner_P = 0
            water_column_immobilizer_C = 0
            water_column_immobilizer_N = 0
            water_column_immobilizer_P = 0

        
 
            #Ecosystem Function Initilization for day zero
            if day == 0:

                #pull in water temp df and daily temperature at the zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus value for the zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus deposition value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value    
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']

            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]            
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]



            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']

            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']

    # Apply Ecosystem Uptake Equations HERE for TN and TP! These update concentrations. ('else' here implies this is now for the rest of the cross-sections in the group)
    #Some values depend on boundary conditions at upstream cross-sections here.
        else:

          #Ecosystem Function Initilization for day zero
            if day == 0:
               #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                 #Pull  in daily water column detritus value for zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,'DetritusC_4'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,'DetritusN_4'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,'DetritusP_4'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero 
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (Set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus deposition value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
              #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']
                
            
            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
           #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]
            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']

            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']




#______SB3_Tributary_____

#New Group_List since a tributary's flow changes stream flow and nutrient mass balance at this part of the stream

#Group List of HEC-RAS cross-sections in the group
group_list = [2782, 2752, 2722, 2692, 2650, 2632, 2602, 2572,2542, 2512, 2482, 2452, 2422, 2392, 2362, 2332,2302, 2272,2242, 2212, 2182,2152, 2122, 2092, 2062, 2032, 2002]

#Model runs iteratively over each day
for day in flow_df.index:
    #Model computes flow hydraulics and ecosystem functions iteratively over each HEC-RAS cross-section in the group_list
    for section in group_list:
        #This group list pans two zones with different ecosystem input data. Sections 2782, 2752, 2722, 2692, 2650, 2632, 2602, 2572 are in zone 4 or reach section 1. Others are in zone K or reach section 2
        if section in [2782, 2752, 2722, 2692, 2650, 2632, 2602, 2572]:
            zone = 4
        else:
            zone = 'K'
    
        #Nutrients N and P and flow from 1st subbasin converted from mg/L to mg/m3 and cfs to cms respectively
        df1_tn_C = sb_outs_df.at[day,'SB3_OUT_N_Conc']*1000
        df1_tp_C = sb_outs_df.at[day,'SB3_OUT_P_Conc']*1000
        df1_Q =  sb_outs_df.at[day,'SB3_Flow']/35.315

        #Nutrients N and P and flow from UPSTREAM converted from mg/L to mg/m3 and cfs to cms respectively
        df2_tn_C = flow_df.at[day,f'{2812} Updated_TN_Conc (mg/m3)']
        df2_tp_C = flow_df.at[day,f'{2812} Updated_TP_Conc (mg/m3)']
        df2_Q = flow_df.at[day,f"{2812} Flow (cms)"]

        #Combine flows from subbasin tributary and UPSTREAM     
        flow_df.at[day,f"{section} Flow (cms)"] = df1_Q + df2_Q

        #Mass balance calculations for N and P from subbasin tributary and UPSTREAM
        dfout_tn_C = (df1_tn_C*df1_Q +df2_tn_C*df2_Q)/(df1_Q + df2_Q)
        dfout_tp_C = (df1_tp_C*df1_Q +df2_tp_C*df2_Q)/(df1_Q + df2_Q)
        
        
        #Start at first HEC-RAS cross section in the group list
        if section == 2782:

            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, cross-sectional area, and approximated benthic area.
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            flow_df.at[day,f'{2782} SWMM_TN_Conc (mg/m3)'] = dfout_tn_C
            flow_df.at[day,f'{2782} SWMM_TP_Conc (mg/m3)'] = dfout_tp_C
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{section} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            # flow_df.at[day,f"{section} Wetted_Perimeter (m)"] = result.at[df_index_len,'Wetted_Perimeter']
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = dfout_tn_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = dfout_tp_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = dfout_tn_C
            tp_conc = dfout_tp_C

        #Apply flow hydraulics interpolation to the rest of HEC-RAS cross-sections in group list
        else:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, and approximated benthic area
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{section} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            # flow_df.at[day,f"{section} Wetted_Perimeter (m)"] = result.at[df_index_len,'Wetted_Perimeter']
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Conc (mg/m3)'] = upstream_tn_conc
            flow_df.at[day,f'{section} SWMM_TP_Conc (mg/m3)'] = upstream_tp_conc
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = upstream_tn_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = upstream_tp_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = upstream_tn_conc
            tp_conc = upstream_tp_conc
        
        #Set water column detritus and microbes to zero at the most upstream part of the group list 
        if section == 2782:
            water_column_detritus_C = 0
            water_column_detritus_N = 0
            water_column_detritus_P = 0
            water_column_miner_C = 0
            water_column_miner_N = 0
            water_column_miner_P = 0
            water_column_immobilizer_C = 0
            water_column_immobilizer_N = 0
            water_column_immobilizer_P = 0


            #Ecosystem Function Initilization for day zero
            if day == 0:

                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero. (set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

 
            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']

            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
           
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]            
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]



            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']

            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']

    # Apply Ecosystem Uptake Equations HERE for TN and TP! These update concentrations. ('else' implies this is now for the rest of the cross-sections in the group)
    #Some values depend on boundary conditions at upstream cross-sections here.
        else:

            #Ecosystem Function Initilization for day zero
            if day == 0:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero. (set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                 #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']

            #Execute all stream ecosystem functions using initialized values  
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]
            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']

            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']



# #______________________ SB5 inflow  _______________________________

#New Group_List since a tributary's flow changes stream flow and nutrient mass balance at this part of the stream
zone = 'K'

#Group List of HEC-RAS cross-sections in the group
group_list = [1972,1942,1912,1882,1852,1822]
#Model runs iteratively over each day
for day in flow_df.index:
    #Model computes flow hydraulics and ecosystem functions iteratively over each HEC-RAS cross-section in the group_list
    for section in group_list:

        #Nutrients N and P and flow from new subbasin tributary converted from mg/L to mg/m3 and cfs to cms respectively
        df1_tn_C = sb_outs_df.at[day,'SB5_OUT_N_Conc']*1000
        df1_tp_C = sb_outs_df.at[day,'SB5_OUT_P_Conc']*1000
        df1_Q =  sb_outs_df.at[day,'SB5_Flow']/35.315

        #Nutrients N and P and flow from UPSTREAM converted from mg/L to mg/m3 and cfs to cms respectively
        df2_tn_C = flow_df.at[day,f'{2002} Updated_TN_Conc (mg/m3)']
        df2_tp_C = flow_df.at[day,f'{2002} Updated_TP_Conc (mg/m3)']
        df2_Q = flow_df.at[day,f"{2002} Flow (cms)"]

       #Combine flows from subbasin 1 and UPSTREAM
        flow_df.at[day,f"{section} Flow (cms)"] = df1_Q + df2_Q

        #Mass balance calculations for N and P from tributary and UPSTREAM
        dfout_tn_C = (df1_tn_C*df1_Q +df2_tn_C*df2_Q)/(df1_Q + df2_Q)
        dfout_tp_C = (df1_tp_C*df1_Q +df2_tp_C*df2_Q)/(df1_Q + df2_Q)

        #Start at first HEC-RAS cross section in the group list
        if section == 1972:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, cross-sectional area, and approximated benthic area.
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            flow_df.at[day,f'{1972} SWMM_TN_Conc (mg/m3)'] = dfout_tn_C
            flow_df.at[day,f'{1972} SWMM_TP_Conc (mg/m3)'] = dfout_tp_C
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = flow_df.at[day,f"{1972} Flow (cms)"]
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = dfout_tn_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = dfout_tp_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = dfout_tn_C
            tp_conc = dfout_tp_C

        #Apply flow hydraulics interpolation to the rest of HEC-RAS cross-sections in group list
        else:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, and approximated benthic area
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{1972} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Conc (mg/m3)'] = upstream_tn_conc
            flow_df.at[day,f'{section} SWMM_TP_Conc (mg/m3)'] = upstream_tp_conc
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = upstream_tn_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = upstream_tp_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = upstream_tn_conc
            tp_conc = upstream_tp_conc
        
        #Set water column detritus and microbes to zero at the most upstream part of the group_list 
        if section == 1972:
            water_column_detritus_C = 0
            water_column_detritus_N = 0
            water_column_detritus_P = 0
            water_column_miner_C = 0
            water_column_miner_N = 0
            water_column_miner_P = 0
            water_column_immobilizer_C = 0
            water_column_immobilizer_N = 0
            water_column_immobilizer_P = 0

        
            #Ecosystem Function Initilization for day zero
            if day == 0:

              #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero. (set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']

            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]            
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]



            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']

            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']


    # Apply Ecosystem Uptake Equations HERE for TN and TP! These update concentrations. ('else' implies this is now for the rest of the cross-sections in the group)
    #Some values depend on boundary conditions at upstream cross-sections here.
        else:

            #Ecosystem Function Initilization for day zero
            if day == 0:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero. (set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']
                
            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]
            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']

            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']


#______________________ SB6 Inflow _______________________________
#New Group_List since a tributary's flow changes stream flow and nutrient mass balance at this part of the stream

#Set zone 
zone = 'K'


#Group List of HEC-RAS cross-sections in the group
group_list = [1626,1596,1566,1536,1506,1476]

#Model runs iteratively over each day
for day in flow_df.index:
    #Model computes flow hydraulics and ecosystem functions iteratively over each HEC-RAS cross-section in the group_list
    for section in group_list:

        #Nutrients N and P and flow from new subbasin tributary converted from mg/L to mg/m3 and cfs to cms respectively
        df1_tn_C = sb_outs_df.at[day,'SB6_OUT_N_Conc']*1000
        df1_tp_C = sb_outs_df.at[day,'SB6_OUT_P_Conc']*1000
        df1_Q =  sb_outs_df.at[day,'SB6_Flow']/35.315

        #Nutrients N and P and flow from UPSTREAM converted from mg/L to mg/m3 and cfs to cms respectively
        df2_tn_C = flow_df.at[day,f'{1822} Updated_TN_Conc (mg/m3)']
        df2_tp_C = flow_df.at[day,f'{1822} Updated_TP_Conc (mg/m3)']
        df2_Q = flow_df.at[day,f"{1822} Flow (cms)"]

        #Combine flows from subbasin 1 and UPSTREAM
        flow_df.at[day,f"{section} Flow (cms)"] = df1_Q + df2_Q

        #Mass balance calculations for N and P from tributary and UPSTREAM
        dfout_tn_C = (df1_tn_C*df1_Q +df2_tn_C*df2_Q)/(df1_Q + df2_Q)
        dfout_tp_C = (df1_tp_C*df1_Q +df2_tp_C*df2_Q)/(df1_Q + df2_Q)
        
        #Start at first HEC-RAS cross section in the group list
        if section == 1626:

            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, cross-sectional area, and approximated benthic area.
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            flow_df.at[day,f'{1626} SWMM_TN_Conc (mg/m3)'] = dfout_tn_C
            flow_df.at[day,f'{1626} SWMM_TP_Conc (mg/m3)'] = dfout_tp_C
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = flow_df.at[day,f"{1626} Flow (cms)"]
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = dfout_tn_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = dfout_tp_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = dfout_tn_C
            tp_conc = dfout_tp_C
            
        #Apply flow hydraulics interpolation to the rest of HEC-RAS cross-sections in group list
        else:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, and approximated benthic area
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = flow_df.at[day,f"{1626} Flow (cms)"]
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model) 
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Conc (mg/m3)'] = upstream_tn_conc
            flow_df.at[day,f'{section} SWMM_TP_Conc (mg/m3)'] = upstream_tp_conc
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = upstream_tn_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = upstream_tp_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = upstream_tn_conc
            tp_conc = upstream_tp_conc
        
        #Set water column detritus and microbes to zero at the most upstream part of the group_list
        if section == 1626:
            water_column_detritus_C = 0
            water_column_detritus_N = 0
            water_column_detritus_P = 0
            water_column_miner_C = 0
            water_column_miner_N = 0
            water_column_miner_P = 0
            water_column_immobilizer_C = 0
            water_column_immobilizer_N = 0
            water_column_immobilizer_P = 0


            #Ecosystem Function Initilization for day zero
            if day == 0:

                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero. (set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']

            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]            
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]



            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']

            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']

    # Apply Ecosystem Uptake Equations HERE for TN and TP! These update concentrations. ('else' implies this is now for the rest of the cross-sections in the group)
    #Some values depend on boundary conditions at upstream cross-sections here.
        else:
            #Ecosystem Function Initilization for day zero
            if day == 0:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero. (set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']
                
            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]
            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']

            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']

#________________________SB7_SB_Inflow_______
#New Group_List since a tributary's flow changes stream flow and nutrient mass balance at this part of the stream
zone = 7

#Group List of HEC-RAS cross-sections in the group
group_list = [1380,1350, 1320, 1290, 1260, 1230, 1200, 1170, 1140, 1110, 1080, 1050, 1020]


#Model runs iteratively over each day
for day in flow_df.index:
    #Model computes flow hydraulics and ecosystem functions iteratively over each HEC-RAS cross-section in the group_list
    for section in group_list:


        #Nutrients N and P and flow from new subbasin tributary converted from mg/L to mg/m3 and cfs to cms respectively
        df1_tn_C = sb_outs_df.at[day,'SB7/8_OUT_N_Conc']*1000
        df1_tp_C = sb_outs_df.at[day,'SB7/8_OUT_P_Conc']*1000
        df1_Q =  sb_outs_df.at[day,'SB7/8_Flow']/35.315


        #Nutrients N and P and flow from UPSTREAM converted from mg/L to mg/m3 and cfs to cms respectively
        df2_tn_C = flow_df.at[day,f'{1476} Updated_TN_Conc (mg/m3)']
        df2_tp_C = flow_df.at[day,f'{1476} Updated_TP_Conc (mg/m3)']
        df2_Q = flow_df.at[day,f"{1476} Flow (cms)"]

        #Combine flows from subbasin 1 and UPSTREAM
        flow_df.at[day,f"{section} Flow (cms)"] = df1_Q + df2_Q

        #Mass balance calculations for N and P from tributary and UPSTREAM
        dfout_tn_C = (df1_tn_C*df1_Q +df2_tn_C*df2_Q)/(df1_Q + df2_Q)
        dfout_tp_C = (df1_tp_C*df1_Q +df2_tp_C*df2_Q)/(df1_Q + df2_Q)

        #Start at first HEC-RAS cross section in the group list
        if section == 1380:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, cross-sectional area, and approximated benthic area.
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            flow_df.at[day,f'{1380} SWMM_TN_Conc (mg/m3)'] = dfout_tn_C
            flow_df.at[day,f'{1380} SWMM_TP_Conc (mg/m3)'] = dfout_tp_C
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{section} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = dfout_tn_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = dfout_tp_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = dfout_tn_C
            tp_conc = dfout_tp_C

        #Apply flow hydraulics interpolation to the rest of HEC-RAS cross-sections in group list
        else:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, and approximated benthic area
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{section} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Conc (mg/m3)'] = upstream_tn_conc
            flow_df.at[day,f'{section} SWMM_TP_Conc (mg/m3)'] = upstream_tp_conc
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = upstream_tn_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = upstream_tp_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = upstream_tn_conc
            tp_conc = upstream_tp_conc
        

        #Set water column detritus and microbes to zero at the most upstream part of the group_list
        if section == 1380:
            water_column_detritus_C = 0
            water_column_detritus_N = 0
            water_column_detritus_P = 0
            water_column_miner_C = 0
            water_column_miner_N = 0
            water_column_miner_P = 0
            water_column_immobilizer_C = 0
            water_column_immobilizer_N = 0
            water_column_immobilizer_P = 0

            #Ecosystem Function Initilization for day zero
            if day == 0:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero. (set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']


            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]            
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]
            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']

        # Apply Ecosystem Uptake Equations HERE for TN and TP! These update concentrations. (else implies this is now for the rest of the cross-sections in the group)
        #Some values depend on boundary conditions at upstream cross-sections here.
        else:

            #Ecosystem Function Initilization for day zero
            if day == 0:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero. (set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,'DetritusC_4'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,'DetritusN_4'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,'DetritusP_4'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']
                
            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]
            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']


# ###______SB9___inflow            
#New Group_List since a tributary's flow changes stream flow and nutrient mass balance at this part of the stream
#Set Zone
zone = 7

#Group List of HEC-RAS cross-sections in the group
group_list = [990, 960, 930, 900, 870, 840, 810, 780, 750, 720, 690, 660, 630, 600, 570, 540, 510, 480, 450, 420, 390, 360, 330, 300]

#Model runs iteratively over each day
for day in flow_df.index:
    #Model computes flow hydraulics and ecosystem functions iteratively over each HEC-RAS cross-section in the group_list
    for section in group_list:

        #Nutrients N and P and flow from new subbasin tributary converted from mg/L to mg/m3 and cfs to cms respectively
        df1_tn_C = sb_outs_df.at[day,'SB9_OUT_N_Conc']*1000
        df1_tp_C = sb_outs_df.at[day,'SB9_OUT_P_Conc']*1000
        df1_Q =  sb_outs_df.at[day,'SB9_Flow']/35.315

        #Nutrients N and P and flow from UPSTREAM converted from mg/L to mg/m3 and cfs to cms respectively        
        df2_tn_C = flow_df.at[day,f'{1020} Updated_TN_Conc (mg/m3)']
        df2_tp_C = flow_df.at[day,f'{1020} Updated_TP_Conc (mg/m3)']
        df2_Q = flow_df.at[day,f"{1020} Flow (cms)"]

        #Combine flows from subbasin 1 and UPSTREAM
        flow_df.at[day,f"{section} Flow (cms)"] = df1_Q + df2_Q

        #Mass balance calculations for N and P from tributary and UPSTREAM
        dfout_tn_C = (df1_tn_C*df1_Q +df2_tn_C*df2_Q)/(df1_Q + df2_Q)
        dfout_tp_C = (df1_tp_C*df1_Q +df2_tp_C*df2_Q)/(df1_Q + df2_Q)

        #Start at first HEC-RAS cross section in the group list
        if section == 990:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, cross-sectional area, and approximated benthic area.
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            flow_df.at[day,f'{990} SWMM_TN_Conc (mg/m3)'] = dfout_tn_C
            flow_df.at[day,f'{990} SWMM_TP_Conc (mg/m3)'] = dfout_tp_C
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{section} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = dfout_tn_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = dfout_tp_C*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = dfout_tn_C
            tp_conc = dfout_tp_C

        #Apply flow hydraulics interpolation to the rest of HEC-RAS cross-sections in group list
        else:
            #HEC-RAS Steady-State Interpolation and conversion of channel flow to velocity, volume, and approximated benthic area
            #Also conversion of nutrient concentrations to nutrient mass in the water column
            section_QtoX = channel_geometry_df.loc[channel_geometry_df['Section'] == section]
            dummy_df = pd.DataFrame(columns = ['Discharge'])
            dummy_df.at[0,'Discharge'] = np.float64(pd.to_numeric(flow_df.at[day,f"{section} Flow (cms)"]))
            result = section_QtoX.append({'Discharge':dummy_df.at[0,'Discharge']}, ignore_index=True)
            result = result.sort_values(by = 'Discharge')
            result = result.interpolate(method ='linear', limit_direction ='forward')
            df_index_len = (len(result)-1)
            flow_df.at[day,f"{section} Benthic_Area (m2)"] = result.at[df_index_len,'Surface_Width']*30  #Benthic area is approximated by multiplying surface width by lenght of each cross-section (30m in this model)
            flow_df.at[day,f"{section} Flow_Area (m2)"] = result.at[df_index_len,'Flow_Area']
            flow_df.at[day,f"{section} Flow_Velocity (m/s)"] = result.at[df_index_len,'Velocity']
            flow_df.at[day,f"{section} Flow_Volume (m3)"] = result.at[df_index_len,'Volume']
            flow_df.at[day,f'{section} SWMM_TN_Conc (mg/m3)'] = upstream_tn_conc
            flow_df.at[day,f'{section} SWMM_TP_Conc (mg/m3)'] = upstream_tp_conc
            flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] = upstream_tn_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] = upstream_tp_conc*flow_df.at[day,f"{section} Flow_Volume (m3)"]
            tn_conc = upstream_tn_conc
            tp_conc = upstream_tp_conc
        
        #Set water column detritus and microbes to zero at the most upstream part of the group_list
        if section == 990:
            water_column_detritus_C = 0
            water_column_detritus_N = 0
            water_column_detritus_P = 0
            water_column_miner_C = 0
            water_column_miner_N = 0
            water_column_miner_P = 0
            water_column_immobilizer_C = 0
            water_column_immobilizer_N = 0
            water_column_immobilizer_P = 0

            #Ecosystem Function Initilization for day zero
            if day == 0:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero. (set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']

            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])

            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]            
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]
            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']

    # Apply Ecosystem Uptake Equations HERE for TN and TP! These update concentrations. ('else' implies this is now for the rest of the cross-sections in the group)
    #Some values depend on boundary conditions at upstream cross-sections here.
        else:

            #Ecosystem Function Initilization for day zero
            if day == 0:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,f'DetritusC_{zone}'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,f'DetritusN_{zone}'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,f'DetritusP_{zone}'] + water_column_detritus_P
                #Set inital benthic algal C,N,and P values. Only for day zero. (set before simulation)
                algal_C_mass = initial_algal_C_mass
                algal_N_mass = algal_C_mass*algal_NC_ratio
                algal_P_mass = algal_C_mass*algal_PC_ratio
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for the section/day
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #For day 0 only, set initial benthic and microbe mass (set before simulation)
                benthic_C_mass = initial_benthic_C
                benthic_N_mass = initial_benthic_N
                benthic_P_mass = initial_benthic_P
                miner_C_mass = initial_miner_C
                miner_N_mass = initial_miner_N
                miner_P_mass = initial_miner_P
                immobilizer_C_mass = initial_immobilizer_C
                immobilizer_N_mass = initial_immobilizer_N
                immobilizer_P_mass = initial_immobilizer_P
                downstream_input_C = detritus_C
                downstream_input_N = detritus_N
                downstream_input_P = detritus_P

            #Ecosystem Function Initilization for days other than zero - inital values depend on previous day's end values
            else:
                #pull in water temp df and daily temperature at appropriate zone
                water_temp_df = daily_temp_df.loc[daily_temp_df['Day']==day]
                water_temp = water_temp_df.at[day,f'TEMP_{zone}']
                #Set algal and detritus Q10f function values based on water temp
                algal_Q10f = algal_Q10**(0.1*(water_temp-1.2))
                detritus_Q10f = detritus_Q10**(0.1*(water_temp - 1.2))
                #Pull  in daily water column detritus depostion value for each zone split into C,N,and P
                daily_detritus_df = detritus_df.loc[detritus_df['Day']==day]
                detritus_C = daily_detritus_df.at[day,'DetritusC_4'] + water_column_detritus_C
                detritus_N = daily_detritus_df.at[day,'DetritusN_4'] + water_column_detritus_N
                detritus_P = daily_detritus_df.at[day,'DetritusP_4'] + water_column_detritus_P
                #Set initial benthic algal mass as the previous day's value  (before entrainment and primary production)
                algal_C_mass = eco_df.at[day-1,f'{section} Algal_C_Mass (mg/m2)']
                algal_N_mass = eco_df.at[day-1,f'{section} Algal_N_Mass (mg/m2)']
                algal_P_mass = eco_df.at[day-1,f'{section} Algal_P_Mass (mg/m2)']
                #Pull in daily PAR value
                light_df = hourly_par_df.loc[hourly_par_df['Day']==day]
                #Set flow hydraulics for other days
                benthic_area = flow_df.at[day,f'{section} Benthic_Area (m2)']
                volume = flow_df.at[day,f'{section} Flow_Volume (m3)']
                velocity = flow_df.at[day,f'{section} Flow_Velocity (m/s)']
                #Set initial benthic detritus and microbe masses as the previous day's value (before entrainment, deposition, and growth)
                benthic_C_mass = eco_df.at[day-1,f'{section} Benthic_C_Mass (mg/m2)']
                benthic_N_mass = eco_df.at[day-1,f'{section} Benthic_N_Mass (mg/m2)']
                benthic_P_mass = eco_df.at[day-1,f'{section} Benthic_P_Mass (mg/m2)']
                miner_C_mass = eco_df.at[day-1,f'{section} Miner_C_Mass (mg/m2)']
                miner_N_mass = eco_df.at[day-1,f'{section} Miner_N_Mass (mg/m2)']
                miner_P_mass = eco_df.at[day-1,f'{section} Miner_P_Mass (mg/m2)']
                immobilizer_C_mass = eco_df.at[day-1,f'{section} Immobilizer_C_Mass (mg/m2)']
                immobilizer_N_mass = eco_df.at[day-1,f'{section} Immobilizer_N_Mass (mg/m2)']
                immobilizer_P_mass = eco_df.at[day-1,f'{section} Immobilizer_P_Mass (mg/m2)']
                
            #Execute all stream ecosystem functions using initialized values
            eco_processes = SSHBS_Uptake_Fxns(day,tn_conc,tp_conc,algal_C_mass,algal_N_mass,algal_P_mass,light_df,water_temp,algal_NC_ratio,algal_PC_ratio,benthic_area,velocity,volume,benthic_C_mass,benthic_N_mass,benthic_P_mass,miner_C_mass,miner_N_mass,miner_P_mass,immobilizer_C_mass,immobilizer_N_mass,immobilizer_P_mass,zone,flow_df.at[day,f"{section} Flow (cms)"])
            
            #Get stream ecosystem function results for each section and update results output files
            eco_df.at[day,f'{section} NPP (mgC/m2)'] = eco_processes[0] 
            eco_df.at[day,f'{section} Algal_N_Uptake (mg/m2)'] = eco_processes[1] 
            eco_df.at[day,f'{section} Algal_P_Uptake (mg/m2)'] = eco_processes[2] 
            eco_df.at[day,f'{section} Algal_C_Mass (mg/m2)'] = eco_processes[3] 
            eco_df.at[day,f'{section} Algal_N_Mass (mg/m2)'] = eco_processes[4] 
            eco_df.at[day,f'{section} Algal_NC_Ratio'] = eco_processes[5] 
            eco_df.at[day,f'{section} Algal_P_Mass (mg/m2)'] = eco_processes[6] 
            eco_df.at[day,f'{section} Algal_PC_Ratio'] = eco_processes[7] 
            eco_df.at[day,f'{section} Algal_C_Mineralization (mg/m2)'] = eco_processes[8]
            eco_df.at[day,f'{section} Algal_N_Mineralization (mg/m2)'] = eco_processes[9]         
            eco_df.at[day,f'{section} Algal_P_Mineralization (mg/m2)'] = eco_processes[10]  
            eco_df.at[day,f'{section} Algal_C_Mortality (mg/m2)'] = eco_processes[11]
            eco_df.at[day,f'{section} Algal_N_Mortality (mg/m2)'] = eco_processes[12]         
            eco_df.at[day,f'{section} Algal_P_Mortality (mg/m2)'] = eco_processes[13] 
            eco_df.at[day,f'{section} Algal_C_Entrained (mg/m2)'] = eco_processes[14]
            eco_df.at[day,f'{section} Algal_N_Entrained (mg/m2)'] = eco_processes[15]         
            eco_df.at[day,f'{section} Algal_P_Entrained (mg/m2)'] = eco_processes[16]   
            eco_df.at[day,f'{section} Benthic_C_Mass (mg/m2)'] = eco_processes[17]
            eco_df.at[day,f'{section} Benthic_N_Mass (mg/m2)'] = eco_processes[18]         
            eco_df.at[day,f'{section} Benthic_P_Mass (mg/m2)'] = eco_processes[19]   
            eco_df.at[day,f'{section} Miner_C_Mass (mg/m2)'] = eco_processes[20]
            eco_df.at[day,f'{section} Miner_N_Mass (mg/m2)'] = eco_processes[21]         
            eco_df.at[day,f'{section} Miner_P_Mass (mg/m2)'] = eco_processes[22]
            eco_df.at[day,f'{section} Miner_N_Mineralization (mg/m2)'] = eco_processes[23]         
            eco_df.at[day,f'{section} Miner_P_Mineralization (mg/m2)'] = eco_processes[24] 
            eco_df.at[day,f'{section} Miner_C_Mortality (mg/m2)'] = eco_processes[25]         
            eco_df.at[day,f'{section} Miner_C_Respiration (mg/m2)'] = eco_processes[26] 
            eco_df.at[day,f'{section} Immobilizer_C_Mass (mg/m2)'] = eco_processes[27]
            eco_df.at[day,f'{section} Immobilizer_N_Mass (mg/m2)'] = eco_processes[28]         
            eco_df.at[day,f'{section} Immobilizer_P_Mass (mg/m2)'] = eco_processes[29]
            eco_df.at[day,f'{section} Immobilizer_N_Mineralization (mg/m2)'] = eco_processes[30]         
            eco_df.at[day,f'{section} Immobilizer_P_Mineralization (mg/m2)'] = eco_processes[31] 
            eco_df.at[day,f'{section} Immobilizer_C_Mortality (mg/m2)'] = eco_processes[32]         
            eco_df.at[day,f'{section} Immobilizer_C_Respiration (mg/m2)'] = eco_processes[33]
            eco_df.at[day,f'{section} Denitrification (mg/m2)'] = eco_processes[34]
            eco_df.at[day,f'{section} Total_N_Mineralization (mg/m2)'] = eco_processes[35]         
            eco_df.at[day,f'{section} Total_P_Mineralization (mg/m2)'] = eco_processes[36] 
            eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)'] = eco_processes[37]         
            eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)'] = eco_processes[38]
            eco_df.at[day,f'{section} Detritus_C_Respiration (mg/m2)'] = eco_processes[39]
            water_column_detritus_C = detritus_C + eco_processes[40] - eco_processes[49]
            eco_df.at[day,f'{section} Water_Column_Detritus_C (mg/m2)'] = water_column_detritus_C
            eco_df.at[day,f'{section} Entrained_Detritus_N (mg/m2)'] = eco_processes[41]
            water_column_detritus_N = detritus_N + eco_processes[41] - eco_processes[50]
            eco_df.at[day,f'{section} Water_Column_Detritus_N (mg/m2)'] = water_column_detritus_N
            eco_df.at[day,f'{section} Entrained_Detritus_P (mg/m2)'] = eco_processes[42]
            water_column_detritus_P = detritus_P + eco_processes[42] - eco_processes[51]
            eco_df.at[day,f'{section} Water_Column_Detritus_P (mg/m2)'] = water_column_detritus_P
            eco_df.at[day,f'{section} Entrained_Miner_C (mg/m2)'] = eco_processes[43]
            water_column_miner_C = water_column_miner_C + eco_processes[43] - eco_processes[52]
            eco_df.at[day,f'{section} Water_Column_Miner_C (mg/m2)'] = water_column_miner_C
            eco_df.at[day,f'{section} Entrained_Miner_N (mg/m2)'] = eco_processes[44]
            water_column_miner_N = water_column_miner_N + eco_processes[44] - eco_processes[53]
            eco_df.at[day,f'{section} Water_Column_Miner_N (mg/m2)'] = water_column_miner_N
            eco_df.at[day,f'{section} Entrained_Miner_P (mg/m2)'] = eco_processes[45]
            water_column_miner_P = water_column_miner_P + eco_processes[45] - eco_processes[54]
            eco_df.at[day,f'{section} Water_Column_Miner_P (mg/m2)'] = water_column_miner_P
            eco_df.at[day,f'{section} Entrained_Immobilizer_C (mg/m2)'] = eco_processes[46]
            water_column_immobilizer_C = water_column_immobilizer_C + eco_processes[46] - eco_processes[55]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_C (mg/m2)'] = water_column_immobilizer_C
            eco_df.at[day,f'{section} Entrained_Immobilizer_N (mg/m2)'] = eco_processes[47]
            water_column_immobilizer_N = water_column_immobilizer_N +eco_processes[47] - eco_processes[56]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_N (mg/m2)'] = water_column_immobilizer_N
            eco_df.at[day,f'{section} Entrained_Immobilizer_P (mg/m2)'] = eco_processes[48]
            water_column_immobilizer_P = water_column_immobilizer_P + eco_processes[48] - eco_processes[57]
            eco_df.at[day,f'{section} Water_Column_Immobilizer_P (mg/m2)'] = water_column_immobilizer_P
            eco_df.at[day,f'{section} Miner_N_Assimilation (mg/m2)'] = eco_processes[58]
            eco_df.at[day,f'{section} Immobilizer_N_Assimilation (mg/m2)'] = eco_processes[59]
            eco_df.at[day,f'{section} Algal_Mass (mg/m2)'] = eco_processes[3] + eco_processes[4] +eco_processes[6]
            eco_df.at[day,f'{section} Benthic_Mass (mg/m2)'] = eco_processes[17] + eco_processes[18] +eco_processes[19]
            eco_df.at[day,f'{section} Total_Respiration (mg/m2)'] = eco_processes[39] + eco_processes[33] +eco_processes[26] +eco_processes[8]
            eco_df.at[day,f'{section} Deposited_Detritus'] = eco_processes[49] + eco_processes[50] + eco_processes[51]
            eco_df.at[day,f'{section} Entrained_Detritus'] = eco_processes[40] + eco_processes[41] + eco_processes[42]
            flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TN_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_N_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)'] = (flow_df.at[day,f'{section} SWMM_TP_Mass (mg)'] - (eco_df.at[day,f'{section} Water_Column_P_Net_Uptake (mg)']/86400))/flow_df.at[day,f'{section} Flow_Volume (m3)']
            upstream_tn_conc = flow_df.at[day,f'{section} Updated_TN_Conc (mg/m3)']
            upstream_tp_conc = flow_df.at[day,f'{section} Updated_TP_Conc (mg/m3)']


#_________________Export Results to CSV files_______

test_date = datetime.datetime.strptime("1/1/2021", "%m/%d/%Y") # Date needs to be set to first day of simulation
K = 730 # Number of days simulated
date_generated = pd.date_range(test_date, periods=K)
flow_df['Date'] = list(date_generated.strftime("%m/%d/%Y"))
flow_df['Date'].astype(str)
flow_df['Date'] = pd.to_datetime(flow_df['Date'], format='%m/%d/%Y')



eco_df.to_csv(r"eco_df_SQ_Test.csv")
flow_df.to_csv(r"flow_df_SQ_Test.csv")